In [ ]:
import time
from pathlib import Path

import numpy as np
import cv2
from mss import mss
import win32gui

In [ ]:
def list_windows():
    def _cb(hwnd, titles):
        title = win32gui.GetWindowText(hwnd)
        if title and win32gui.IsWindowVisible(hwnd):
            titles.append(title)
    titles = []
    win32gui.EnumWindows(_cb, titles)
    print("Visible windows:")
    for t in titles:
        print("  ", t)

In [ ]:
def get_emuhawk_rect():
    # Try whatever title BizHawk is showing on your machine:
    # common ones are "EmuHawk", "BizHawk" or the ROM name + " - EmuHawk"
    hwnd = win32gui.FindWindow(None, "EmuHawk")
    if not hwnd:
        raise RuntimeError("EmuHawk window not found – check the exact title above")
    left, top, right, bottom = win32gui.GetWindowRect(hwnd)
    return left, top, right-left, bottom-top

In [ ]:
list_windows()
try:
    x, y, w, h = get_emuhawk_rect()
    print(f"\nFound EmuHawk at ({x}, {y}), size {w}×{h}")
except Exception as e:
    print("\nError:", e)

In [ ]:
import cv2
import numpy as np
from mss import mss
import win32gui

WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"

def get_bro2_rect():
    hwnd = win32gui.FindWindow(None, WINDOW_TITLE)
    if not hwnd:
        raise RuntimeError(f"Window not found: {WINDOW_TITLE!r}")
    l, t, r, b = win32gui.GetWindowRect(hwnd)
    return {"left": l, "top": t, "width": r-l, "height": b-t}

# 1) grab a single screenshot
sct   = mss()
bbox  = get_bro2_rect()
frame = np.array(sct.grab(bbox))[:, :, :3]   # BGRA→BGR

# 2) let you draw the P1‐health bbox
cv2.namedWindow("Select P1 Health Bar", cv2.WINDOW_NORMAL)
roi = cv2.selectROI("Select P1 Health Bar", frame, showCrosshair=True, fromCenter=False)
cv2.destroyAllWindows()

x,y,w,h = roi
print(f"P1 health bar ROI (x, y, w, h): ({x}, {y}, {w}, {h})")


P1 health bar ROI (x, y, w, h): (518, 193, 407, 33)
P2 health bar ROI (x, y, w, h): (1032, 193, 406, 32)

A1 health bar ROI (x, y, w, h): (518, 868, 194, 29)
Animal2 health bar ROI (x, y, w, h): (1243, 866, 196, 36)

In [ ]:
import time
import numpy as np
from mss import mss
import win32gui

# PS 1  500, 1000, 155

# ——— CONFIG ———
WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"
X_START, X_END, Y = 500, 1000, 155
WIDTH  = X_END - X_START + 1  # 501 pixels
HEIGHT = 1
N      = 200                   # number of iterations for the benchmark

# 1) Find the window and compute client‐area origin
hwnd = win32gui.FindWindow(None, WINDOW_TITLE)
if not hwnd:
    raise RuntimeError(f"Window not found: {WINDOW_TITLE!r}")
left, top = win32gui.ClientToScreen(hwnd, (0, 0))

# 2) Build the ROI in absolute screen coords
roi = {
    "left":   left  + X_START,
    "top":    top   + Y,
    "width":  WIDTH,
    "height": HEIGHT,
}

sct    = mss()
times  = np.zeros(N, dtype=float)

# 3) Benchmark: grab the 501×1 strip and optionally index any points
for i in range(N):
    t0 = time.perf_counter()
    
    raw   = sct.grab(roi)                     # BGRA buffer
    frame = np.array(raw)[:, :, :3]           # → (1, 501, 3) BGR
    
    # Example: sample your full list of SAMPLES = [(x,155) for x in range(500,1001)]
    # rel_x = x - X_START; rel_y = 0
    # values = [tuple(frame[0, x - X_START]) for x in range(X_START, X_END+1)]
    # (we skip explicit sampling here since you only care about speed)
    
    times[i] = (time.perf_counter() - t0) * 1000  # ms

# 4) Compute stats
avg_ms = times.mean()
min_ms = times.min()
max_ms = times.max()
fps    = 1000.0 / avg_ms

print(f"Ran {N} grabs of {WIDTH}×{HEIGHT} strip at y={Y}")
print(f"  Avg time: {avg_ms:.2f} ms  |  Min: {min_ms:.2f} ms  |  Max: {max_ms:.2f} ms")
print(f"  Equivalent FPS: {fps:.1f}")


In [ ]:
#!/usr/bin/env python3
import time
import numpy as np
from mss import mss
import win32gui

# ——— CONFIG ———
WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"
# sample coords RELATIVE to the client-area top-left

PLAYER_1_X_START = 500
PLAYER_1_X_END   = 1000
PLAYER_1_Y       = 155

PLAYER_2_X_START = 950
PLAYER_2_X_END   = 1800 # 155   
PLAYER_1_Y       = 155

SAMPLES = [ (x, 155) for x in range(PLAYER_2_X_START, PLAYER_2_X_END + 1) ]  # 501 samples at y=155
   

def get_client_bbox(hwnd):
    # returns (left, top, width, height) of client-area in SCREEN coords
    cx0, cy0, cx1, cy1 = win32gui.GetClientRect(hwnd)
    width, height = cx1 - cx0, cy1 - cy0
    left, top = win32gui.ClientToScreen(hwnd, (0, 0))
    return left, top, width, height

def classify(rgb):
    r, g, b = rgb
    if   r > 200 and g < 100 and b < 100:
        return "red-ish"
    elif r > 200 and g > 200 and b < 100:
        return "yellow-ish"
    elif r <  50 and g <  50 and b <  50:
        return "black"
    else:
        return "other"

if __name__ == "__main__":
    # 1) locate the window
    hwnd = win32gui.FindWindow(None, WINDOW_TITLE)
    if not hwnd:
        raise RuntimeError(f"Cannot find window titled {WINDOW_TITLE!r}")

    # 2) compute client bbox
    left, top, w, h = get_client_bbox(hwnd)
    roi = {"left": left, "top": top, "width": w, "height": h}

    sct = mss()
    fps = 10
    interval = 1.0 / fps

    print(f"Sampling {len(SAMPLES)} points in '{WINDOW_TITLE}' at ~{fps} Hz")
    print("Ctrl-C to stop.\n")

    try:
        while True:
            t0 = time.time()
            # grab one frame of the client area (BGRA)
            frame = np.array(sct.grab(roi))[:, :, :3]  # → (H, W, 3) BGR

            for x_rel, y_rel in SAMPLES:
                b, g, r = frame[y_rel, x_rel]
                rgb      = (int(r), int(g), int(b))
                hexcode  = f"#{r:02x}{g:02x}{b:02x}"
                name     = classify(rgb)
                print(f"({x_rel:3d},{y_rel:3d}) → RGB={rgb}  HEX={hexcode}  ⇒ {name}")

            print("-" * 60)
            # pace the loop
            dt = time.time() - t0
            if dt < interval:
                time.sleep(interval - dt)

    except KeyboardInterrupt:
        print("\nStopped sampling.")


In [ ]:
import time
import numpy as np
from mss import mss
import win32gui

def monitor_health(window_title, x_start, bar_len, y, fps,
                   lower_bgr, upper_bgr, drop_per_px):
    """
    Live health bar monitor.

    Parameters:
      window_title (str): Title of the window to capture
      x_start (int): X offset of the bar's left edge in client coordinates
      bar_len (int): Length of the bar in pixels
      y (int): Y offset of the bar in client coordinates
      fps (int): Frames per second to capture
      lower_bgr (np.ndarray): Lower BGR threshold array [B, G, R]
      upper_bgr (np.ndarray): Upper BGR threshold array [B, G, R]
      drop_per_px (float): Percentage drop per pixel (e.g., 0.25 for ¼%)
    """
    # Locate the window
    hwnd = win32gui.FindWindow(None, window_title)
    if not hwnd:
        raise RuntimeError(f"Window not found: {window_title!r}")
    left, top = win32gui.ClientToScreen(hwnd, (0, 0))
    roi = {
        'left':  left + x_start,
        'top':   top  + y,
        'width':  bar_len,
        'height': 1,
    }

    sct = mss()
    interval = 1.0 / fps
    print("Starting live health monitor (Ctrl-C to stop)\n")

    try:
        while True:
            t0 = time.perf_counter()

            # Capture the bar strip and split channels
            raw   = sct.grab(roi)
            strip = np.array(raw)[:, :, :3]   # shape=(1,bar_len,3)
            b, g, r = strip[0].T              # each length=bar_len

            # Build a mask of 'yellow' pixels
            mask = (
                (r >= lower_bgr[2]) & (r <= upper_bgr[2]) &
                (g >= lower_bgr[1]) & (g <= upper_bgr[1]) &
                (b >= lower_bgr[0]) & (b <= upper_bgr[0])
            )

            # Find last non-yellow index
            non_yellow = np.nonzero(~mask)[0]
            last_idx = non_yellow.max() if non_yellow.size else -1

            # Compute life percentage
            drop_pixels = max(0, last_idx + 1)
            life_pct = 100.0 - (drop_pixels * drop_per_px)
            life_pct = np.clip(life_pct, 0.0, 100.0)

            # Print results
            if last_idx < 0:
                print("No non-yellow pixel detected → life = 100.00%\n")
            else:
                x_coord = x_start + last_idx
                print(f"Last non-yellow at x = {x_coord}")
                for rel in (-1, 0, +1):
                    idx = last_idx + rel
                    if 0 <= idx < bar_len:
                        R, G, B = int(r[idx]), int(g[idx]), int(b[idx])
                        print(f"  rel {rel:+} (x={x_start+idx}): RGB=({R:3d},{G:3d},{B:3d})")
                print(f"Computed life: {life_pct:6.2f}%\n")

            # Throttle to target FPS
            dt = time.perf_counter() - t0
            if dt < interval:
                time.sleep(interval - dt)

    except KeyboardInterrupt:
        print("\nStopped.")


if __name__ == "__main__":
    # Default configuration
    WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"
    PLAYER_1_X = 505
    PLAYER_2_X = 1021
    BAR_LEN = 400
    Y = 155
    FPS = 30
    LOWER_BGR = np.array([  0, 160, 190], dtype=np.uint8)
    UPPER_BGR = np.array([ 20, 180, 220], dtype=np.uint8)
    DROP_PER_PX = 0.25

    monitor_health(
        WINDOW_TITLE,
        PLAYER_1_X,
        BAR_LEN,
        Y,
        FPS,
        LOWER_BGR,
        UPPER_BGR,
        DROP_PER_PX
    )

    monitor_health(
        WINDOW_TITLE,
        PLAYER_1_X,
        BAR_LEN,
        Y,
        FPS,
        LOWER_BGR,
        UPPER_BGR,
        DROP_PER_PX
    )


In [ ]:
import time
import numpy as np
from mss import mss
import win32gui

def monitor_health(window_title, x_start, bar_len, y, fps, lower_bgr, upper_bgr, drop_per_px, reverse=False):
    """
    Live health bar monitor.
    
    Parameters:
        window_title (str): Title of the window to capture
        x_start (int): X offset of the bar's left edge in client coordinates
        bar_len (int): Length of the bar in pixels
        y (int): Y offset of the bar in client coordinates
        fps (int): Frames per second to capture
        lower_bgr (np.ndarray): Lower BGR threshold array [B, G, R]
        upper_bgr (np.ndarray): Upper BGR threshold array [B, G, R]
        drop_per_px (float): Percentage drop per pixel (e.g., 0.25 for ¼%)
        reverse (bool): If True, bar depletes from right to left (for Player 2)
    """
    # Locate the window
    hwnd = win32gui.FindWindow(None, window_title)
    if not hwnd:
        raise RuntimeError(f"Window not found: {window_title!r}")
    
    left, top = win32gui.ClientToScreen(hwnd, (0, 0))
    roi = {
        'left': left + x_start,
        'top': top + y,
        'width': bar_len,
        'height': 1,
    }
    
    sct = mss()
    interval = 1.0 / fps
    player = "Player 2" if reverse else "Player 1"
    
    print(f"Starting live health monitor for {player} (Ctrl-C to stop)\n")
    
    try:
        while True:
            t0 = time.perf_counter()
            
            # Capture the bar strip and split channels
            raw = sct.grab(roi)
            strip = np.array(raw)[:, :, :3]  # shape=(1,bar_len,3)
            b, g, r = strip[0].T  # each length=bar_len
            
            # Build a mask of 'yellow' pixels
            mask = (
                (r >= lower_bgr[2]) & (r <= upper_bgr[2]) &
                (g >= lower_bgr[1]) & (g <= upper_bgr[1]) &
                (b >= lower_bgr[0]) & (b <= upper_bgr[0])
            )
            
            # Find last non-yellow index based on direction
            non_yellow = np.nonzero(~mask)[0]
            
            if reverse:  # Player 2: bar depletes from right to left
                # For Player 2, we want the first non-yellow pixel from the left
                last_idx = non_yellow.min() if non_yellow.size else bar_len
                drop_pixels = bar_len - last_idx
            else:  # Player 1: bar depletes from left to right
                # For Player 1, we want the last non-yellow pixel from the left
                last_idx = non_yellow.max() if non_yellow.size else -1
                drop_pixels = max(0, last_idx + 1)
            
            # Compute life percentage
            life_pct = 100.0 - (drop_pixels * drop_per_px)
            life_pct = np.clip(life_pct, 0.0, 100.0)
            
            # Print results
            print(f"{player}:")
            if (not reverse and last_idx < 0) or (reverse and last_idx >= bar_len):
                print("  No non-yellow pixel detected → life = 100.00%\n")
            else:
                x_coord = x_start + last_idx
                print(f"  Last non-yellow at x = {x_coord}")
                for rel in (-1, 0, +1):
                    idx = last_idx + rel
                    if 0 <= idx < bar_len:
                        R, G, B = int(r[idx]), int(g[idx]), int(b[idx])
                        print(f"    rel {rel:+} (x={x_start+idx}): RGB=({R:3d},{G:3d},{B:3d})")
                print(f"  Computed life: {life_pct:6.2f}%\n")
            
            # Throttle to target FPS
            dt = time.perf_counter() - t0
            if dt < interval:
                time.sleep(interval - dt)
                
    except KeyboardInterrupt:
        print(f"\n{player} monitor stopped.")

def monitor_both_players(window_title, p1_x, p2_x, bar_len, y, fps, lower_bgr, upper_bgr, drop_per_px):
    """Monitor both players simultaneously using threading."""
    import threading
    
    # Create threads for each player
    p1_thread = threading.Thread(
        target=monitor_health,
        args=(window_title, p1_x, bar_len, y, fps, lower_bgr, upper_bgr, drop_per_px, False)
    )
    
    p2_thread = threading.Thread(
        target=monitor_health,
        args=(window_title, p2_x - bar_len, bar_len, y, fps, lower_bgr, upper_bgr, drop_per_px, True)
    )
    
    # Start both threads
    p1_thread.start()
    p2_thread.start()
    
    # Wait for both to complete
    p1_thread.join()
    p2_thread.join()

if __name__ == "__main__":
    # Default configuration
    WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"
    PLAYER_1_X = 500      # P1 bar starts at 505
    PLAYER_2_X = 1421     # P2 bar starts at 1421 (but we'll calculate from 1021)
    BAR_LEN = 400
    Y = 155
    FPS = 30
    LOWER_BGR = np.array([0, 160, 190], dtype=np.uint8)
    UPPER_BGR = np.array([20, 180, 220], dtype=np.uint8)
    DROP_PER_PX = 0.25
    
    # Monitor both players simultaneously
    # monitor_both_players(
    #     WINDOW_TITLE,
    #     PLAYER_1_X,
    #     PLAYER_2_X,
    #     BAR_LEN,
    #     Y,
    #     FPS,
    #     LOWER_BGR,
    #     UPPER_BGR,
    #     DROP_PER_PX
    # )
    
    # Or monitor individually:
    # Player 1 only:
    # monitor_health(WINDOW_TITLE, PLAYER_1_X, BAR_LEN, Y, FPS, LOWER_BGR, UPPER_BGR, DROP_PER_PX, False)
    
    # Player 2 only:
    monitor_health(WINDOW_TITLE, 1021, BAR_LEN, Y, FPS, LOWER_BGR, UPPER_BGR, DROP_PER_PX, True)

In [ ]:
import time
import numpy as np
from mss import mss
import win32gui

def monitor_health(window_title, x_start, bar_len, y, fps, lower_bgr, upper_bgr, drop_per_px, reverse=False):
    """
    Live health bar monitor.
    
    Parameters:
        window_title (str): Title of the window to capture
        x_start (int): X offset of the bar's left edge in client coordinates
        bar_len (int): Length of the bar in pixels
        y (int): Y offset of the bar in client coordinates
        fps (int): Frames per second to capture
        lower_bgr (np.ndarray): Lower BGR threshold array [B, G, R]
        upper_bgr (np.ndarray): Upper BGR threshold array [B, G, R]
        drop_per_px (float): Percentage drop per pixel (e.g., 0.25 for ¼%)
        reverse (bool): If True, bar depletes from right to left (for Player 2)
    """
    # Locate the window
    hwnd = win32gui.FindWindow(None, window_title)
    if not hwnd:
        raise RuntimeError(f"Window not found: {window_title!r}")
    
    left, top = win32gui.ClientToScreen(hwnd, (0, 0))
    roi = {
        'left': left + x_start,
        'top': top + y,
        'width': bar_len,
        'height': 1,
    }
    
    sct = mss()
    interval = 1.0 / fps
    player = "Player 2" if reverse else "Player 1"
    
    print(f"Starting live health monitor for {player} (Ctrl-C to stop)\n")
    
    try:
        while True:
            t0 = time.perf_counter()
            
            # Capture the bar strip and split channels
            raw = sct.grab(roi)
            strip = np.array(raw)[:, :, :3]  # shape=(1,bar_len,3)
            b, g, r = strip[0].T  # each length=bar_len
            
            # Build a mask of 'yellow' pixels
            mask = (
                (r >= lower_bgr[2]) & (r <= upper_bgr[2]) &
                (g >= lower_bgr[1]) & (g <= upper_bgr[1]) &
                (b >= lower_bgr[0]) & (b <= upper_bgr[0])
            )
            
            # Find last non-yellow index based on direction
            non_yellow = np.nonzero(~mask)[0]
            
            if reverse:  # Player 2: bar depletes from right to left
                # For Player 2, we want the first non-yellow pixel from the left
                last_idx = non_yellow.min() if non_yellow.size else bar_len
                drop_pixels = bar_len - last_idx
            else:  # Player 1: bar depletes from left to right
                # For Player 1, we want the last non-yellow pixel from the left
                last_idx = non_yellow.max() if non_yellow.size else -1
                drop_pixels = max(0, last_idx + 1)
            
            # Compute life percentage
            life_pct = 100.0 - (drop_pixels * drop_per_px)
            life_pct = np.clip(life_pct, 0.0, 100.0)
            
            # Print results
            print(f"{player}:")
            if (not reverse and last_idx < 0) or (reverse and last_idx >= bar_len):
                print("  No non-yellow pixel detected → life = 100.00%\n")
                life_pct = 100.0
            else:
                x_coord = x_start + last_idx
                print(f"  Last non-yellow at x = {x_coord}")
                for rel in (-1, 0, +1):
                    idx = last_idx + rel
                    if 0 <= idx < bar_len:
                        R, G, B = int(r[idx]), int(g[idx]), int(b[idx])
                        print(f"    rel {rel:+} (x={x_start+idx}): RGB=({R:3d},{G:3d},{B:3d})")
                print(f"  Computed life: {life_pct:6.2f}%\n")
            
            # Write to file for external display
            filename = "player2_health.txt" if reverse else "player1_health.txt"
            with open(filename, "w") as f:
                f.write(f"{life_pct:.2f}")
            
            # Throttle to target FPS
            dt = time.perf_counter() - t0
            if dt < interval:
                time.sleep(interval - dt)
                
    except KeyboardInterrupt:
        print(f"\n{player} monitor stopped.")

def monitor_both_players(window_title, p1_x, p2_x, bar_len, y, fps, lower_bgr, upper_bgr, drop_per_px):
    """Monitor both players in a single thread."""
    # Locate the window
    hwnd = win32gui.FindWindow(None, window_title)
    if not hwnd:
        raise RuntimeError(f"Window not found: {window_title!r}")
    
    left, top = win32gui.ClientToScreen(hwnd, (0, 0))
    
    # Define ROIs for both players
    roi_p1 = {
        'left': left + p1_x,
        'top': top + y,
        'width': bar_len,
        'height': 1,
    }
    
    roi_p2 = {
        'left': left + (p2_x - bar_len),
        'top': top + y,
        'width': bar_len,
        'height': 1,
    }
    
    sct = mss()
    interval = 1.0 / fps
    
    print("Starting live health monitor for both players (Ctrl-C to stop)\n")
    
    try:
        while True:
            t0 = time.perf_counter()
            
            # Capture Player 1
            raw_p1 = sct.grab(roi_p1)
            strip_p1 = np.array(raw_p1)[:, :, :3]
            b1, g1, r1 = strip_p1[0].T
            
            # Capture Player 2
            raw_p2 = sct.grab(roi_p2)
            strip_p2 = np.array(raw_p2)[:, :, :3]
            b2, g2, r2 = strip_p2[0].T
            
            # Process Player 1 (left to right)
            mask_p1 = (
                (r1 >= lower_bgr[2]) & (r1 <= upper_bgr[2]) &
                (g1 >= lower_bgr[1]) & (g1 <= upper_bgr[1]) &
                (b1 >= lower_bgr[0]) & (b1 <= upper_bgr[0])
            )
            non_yellow_p1 = np.nonzero(~mask_p1)[0]
            last_idx_p1 = non_yellow_p1.max() if non_yellow_p1.size else -1
            drop_pixels_p1 = max(0, last_idx_p1 + 1)
            life_pct_p1 = 100.0 - (drop_pixels_p1 * drop_per_px)
            life_pct_p1 = np.clip(life_pct_p1, 0.0, 100.0)
            
            # Process Player 2 (right to left)
            mask_p2 = (
                (r2 >= lower_bgr[2]) & (r2 <= upper_bgr[2]) &
                (g2 >= lower_bgr[1]) & (g2 <= upper_bgr[1]) &
                (b2 >= lower_bgr[0]) & (b2 <= upper_bgr[0])
            )
            non_yellow_p2 = np.nonzero(~mask_p2)[0]
            last_idx_p2 = non_yellow_p2.min() if non_yellow_p2.size else bar_len
            drop_pixels_p2 = bar_len - last_idx_p2
            life_pct_p2 = 100.0 - (drop_pixels_p2 * drop_per_px)
            life_pct_p2 = np.clip(life_pct_p2, 0.0, 100.0)
            
            # Print results
            print(f"Player 1: {life_pct_p1:6.2f}%  |  Player 2: {life_pct_p2:6.2f}%")
            
            # Throttle to target FPS
            dt = time.perf_counter() - t0
            if dt < interval:
                time.sleep(interval - dt)
                
    except KeyboardInterrupt:
        print("\nStopped.")

if __name__ == "__main__":
    # Default configuration
    WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"
    PLAYER_1_X = 505      # P1 bar starts at 505
    PLAYER_2_X = 1421     # P2 bar starts at 1421 (but we'll calculate from 1021)
    BAR_LEN = 400
    Y = 155
    FPS = 30
    LOWER_BGR = np.array([0, 160, 190], dtype=np.uint8)
    UPPER_BGR = np.array([20, 180, 220], dtype=np.uint8)
    DROP_PER_PX = 0.25
    
    # Monitor both players simultaneously
    monitor_both_players(
        WINDOW_TITLE,
        PLAYER_1_X,
        PLAYER_2_X,
        BAR_LEN,
        Y,
        FPS,
        LOWER_BGR,
        UPPER_BGR,
        DROP_PER_PX
    )
    
    

In [ ]:
import time
import numpy as np
from mss import mss
import win32gui
import cv2

def monitor_health(window_title, x_start, bar_len, y, fps, lower_bgr, upper_bgr, drop_per_px, reverse=False):
    """
    Live health bar monitor.
    
    Parameters:
        window_title (str): Title of the window to capture
        x_start (int): X offset of the bar's left edge in client coordinates
        bar_len (int): Length of the bar in pixels
        y (int): Y offset of the bar in client coordinates
        fps (int): Frames per second to capture
        lower_bgr (np.ndarray): Lower BGR threshold array [B, G, R]
        upper_bgr (np.ndarray): Upper BGR threshold array [B, G, R]
        drop_per_px (float): Percentage drop per pixel (e.g., 0.25 for ¼%)
        reverse (bool): If True, bar depletes from right to left (for Player 2)
    """
    # Locate the window
    hwnd = win32gui.FindWindow(None, window_title)
    if not hwnd:
        raise RuntimeError(f"Window not found: {window_title!r}")
    
    left, top = win32gui.ClientToScreen(hwnd, (0, 0))
    roi = {
        'left': left + x_start,
        'top': top + y,
        'width': bar_len,
        'height': 1,
    }
    
    sct = mss()
    interval = 1.0 / fps
    player = "Player 2" if reverse else "Player 1"
    
    print(f"Starting live health monitor for {player} (Ctrl-C to stop)\n")
    
    try:
        while True:
            t0 = time.perf_counter()
            
            # Capture the bar strip and split channels
            raw = sct.grab(roi)
            strip = np.array(raw)[:, :, :3]  # shape=(1,bar_len,3)
            b, g, r = strip[0].T  # each length=bar_len
            
            # Build a mask of 'yellow' pixels
            mask = (
                (r >= lower_bgr[2]) & (r <= upper_bgr[2]) &
                (g >= lower_bgr[1]) & (g <= upper_bgr[1]) &
                (b >= lower_bgr[0]) & (b <= upper_bgr[0])
            )
            
            # Find last non-yellow index based on direction
            non_yellow = np.nonzero(~mask)[0]
            
            if reverse:  # Player 2: bar depletes from right to left
                # For Player 2, we want the first non-yellow pixel from the left
                last_idx = non_yellow.min() if non_yellow.size else bar_len
                drop_pixels = bar_len - last_idx
            else:  # Player 1: bar depletes from left to right
                # For Player 1, we want the last non-yellow pixel from the left
                last_idx = non_yellow.max() if non_yellow.size else -1
                drop_pixels = max(0, last_idx + 1)
            
            # Compute life percentage
            life_pct = 100.0 - (drop_pixels * drop_per_px)
            life_pct = np.clip(life_pct, 0.0, 100.0)
            
            # Print results
            print(f"{player}:")
            if (not reverse and last_idx < 0) or (reverse and last_idx >= bar_len):
                print("  No non-yellow pixel detected → life = 100.00%\n")
                life_pct = 100.0
            else:
                x_coord = x_start + last_idx
                print(f"  Last non-yellow at x = {x_coord}")
                for rel in (-1, 0, +1):
                    idx = last_idx + rel
                    if 0 <= idx < bar_len:
                        R, G, B = int(r[idx]), int(g[idx]), int(b[idx])
                        print(f"    rel {rel:+} (x={x_start+idx}): RGB=({R:3d},{G:3d},{B:3d})")
                print(f"  Computed life: {life_pct:6.2f}%\n")
            
            # Write to file for external display
            filename = "player2_health.txt" if reverse else "player1_health.txt"
            with open(filename, "w") as f:
                f.write(f"{life_pct:.2f}")
            
            # Throttle to target FPS
            dt = time.perf_counter() - t0
            if dt < interval:
                time.sleep(interval - dt)
                
    except KeyboardInterrupt:
        print(f"\n{player} monitor stopped.")

def monitor_both_players(window_title, p1_x, p2_x, bar_len, y, fps, lower_bgr, upper_bgr, drop_per_px):
    """Monitor both players in a single thread."""
    # Locate the window
    hwnd = win32gui.FindWindow(None, window_title)
    if not hwnd:
        raise RuntimeError(f"Window not found: {window_title!r}")
    
    left, top = win32gui.ClientToScreen(hwnd, (0, 0))
    
    # Define ROIs for both players
    roi_p1 = {
        'left': left + p1_x,
        'top': top + y,
        'width': bar_len,
        'height': 1,
    }
    
    roi_p2 = {
        'left': left + (p2_x - bar_len),
        'top': top + y,
        'width': bar_len,
        'height': 1,
    }
    
    sct = mss()
    interval = 1.0 / fps
    
    print("Starting live health monitor for both players (Ctrl-C to stop)\n")
    
    try:
        while True:
            t0 = time.perf_counter()
            
            # Capture Player 1
            raw_p1 = sct.grab(roi_p1)
            strip_p1 = np.array(raw_p1)[:, :, :3]
            b1, g1, r1 = strip_p1[0].T
            
            # Capture Player 2
            raw_p2 = sct.grab(roi_p2)
            strip_p2 = np.array(raw_p2)[:, :, :3]
            b2, g2, r2 = strip_p2[0].T
            
            # Process Player 1 (left to right)
            mask_p1 = (
                (r1 >= lower_bgr[2]) & (r1 <= upper_bgr[2]) &
                (g1 >= lower_bgr[1]) & (g1 <= upper_bgr[1]) &
                (b1 >= lower_bgr[0]) & (b1 <= upper_bgr[0])
            )
            non_yellow_p1 = np.nonzero(~mask_p1)[0]
            last_idx_p1 = non_yellow_p1.max() if non_yellow_p1.size else -1
            drop_pixels_p1 = max(0, last_idx_p1 + 1)
            life_pct_p1 = 100.0 - (drop_pixels_p1 * drop_per_px)
            life_pct_p1 = np.clip(life_pct_p1, 0.0, 100.0)
            
            # Process Player 2 (right to left)
            mask_p2 = (
                (r2 >= lower_bgr[2]) & (r2 <= upper_bgr[2]) &
                (g2 >= lower_bgr[1]) & (g2 <= upper_bgr[1]) &
                (b2 >= lower_bgr[0]) & (b2 <= upper_bgr[0])
            )
            non_yellow_p2 = np.nonzero(~mask_p2)[0]
            last_idx_p2 = non_yellow_p2.min() if non_yellow_p2.size else bar_len
            drop_pixels_p2 = bar_len - last_idx_p2
            life_pct_p2 = 100.0 - (drop_pixels_p2 * drop_per_px)
            life_pct_p2 = np.clip(life_pct_p2, 0.0, 100.0)
            
            # Print results
            print(f"Player 1: {life_pct_p1:6.2f}%  |  Player 2: {life_pct_p2:6.2f}%")
            
            # Throttle to target FPS
            dt = time.perf_counter() - t0
            if dt < interval:
                time.sleep(interval - dt)
                
    except KeyboardInterrupt:
        print("\nStopped.")

def monitor_both_players_with_overlay(window_title, p1_x, p2_x, bar_len, y, fps, lower_bgr, upper_bgr, drop_per_px):
    """Monitor both players with visual overlay."""
    # Locate the window
    hwnd = win32gui.FindWindow(None, window_title)
    if not hwnd:
        raise RuntimeError(f"Window not found: {window_title!r}")
    
    # Get window dimensions
    rect = win32gui.GetClientRect(hwnd)
    width = rect[2] - rect[0]
    height = rect[3] - rect[1]
    left, top = win32gui.ClientToScreen(hwnd, (0, 0))
    
    # Define capture regions
    full_window = {
        'left': left,
        'top': top,
        'width': width,
        'height': height,
    }
    
    roi_p1 = {
        'left': left + p1_x,
        'top': top + y,
        'width': bar_len,
        'height': 1,
    }
    
    roi_p2 = {
        'left': left + (p2_x - bar_len),
        'top': top + y,
        'width': bar_len,
        'height': 1,
    }
    
    sct = mss()
    interval = 1.0 / fps
    
    # Create window for display
    cv2.namedWindow('Health Monitor', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Health Monitor', width // 2, height // 2)
    
    print("Health monitor with overlay running. Press 'q' to quit, 'f' for fullscreen.\n")
    
    try:
        while True:
            t0 = time.perf_counter()
            
            # Capture full window
            full_capture = sct.grab(full_window)
            full_img = np.array(full_capture)[:, :, :3]  # Remove alpha channel
            full_img = cv2.cvtColor(full_img, cv2.COLOR_RGB2BGR)  # Convert RGB to BGR for OpenCV
            
            # Capture Player 1
            raw_p1 = sct.grab(roi_p1)
            strip_p1 = np.array(raw_p1)[:, :, :3]
            b1, g1, r1 = strip_p1[0].T
            
            # Capture Player 2
            raw_p2 = sct.grab(roi_p2)
            strip_p2 = np.array(raw_p2)[:, :, :3]
            b2, g2, r2 = strip_p2[0].T
            
            # Process Player 1 (left to right)
            mask_p1 = (
                (r1 >= lower_bgr[2]) & (r1 <= upper_bgr[2]) &
                (g1 >= lower_bgr[1]) & (g1 <= upper_bgr[1]) &
                (b1 >= lower_bgr[0]) & (b1 <= upper_bgr[0])
            )
            non_yellow_p1 = np.nonzero(~mask_p1)[0]
            last_idx_p1 = non_yellow_p1.max() if non_yellow_p1.size else -1
            drop_pixels_p1 = max(0, last_idx_p1 + 1)
            life_pct_p1 = 100.0 - (drop_pixels_p1 * drop_per_px)
            life_pct_p1 = np.clip(life_pct_p1, 0.0, 100.0)
            
            # Process Player 2 (right to left)
            mask_p2 = (
                (r2 >= lower_bgr[2]) & (r2 <= upper_bgr[2]) &
                (g2 >= lower_bgr[1]) & (g2 <= upper_bgr[1]) &
                (b2 >= lower_bgr[0]) & (b2 <= upper_bgr[0])
            )
            non_yellow_p2 = np.nonzero(~mask_p2)[0]
            last_idx_p2 = non_yellow_p2.min() if non_yellow_p2.size else bar_len
            drop_pixels_p2 = bar_len - last_idx_p2
            life_pct_p2 = 100.0 - (drop_pixels_p2 * drop_per_px)
            life_pct_p2 = np.clip(life_pct_p2, 0.0, 100.0)
            
            # Print to console
            print(f"\rPlayer 1: {life_pct_p1:6.2f}%  |  Player 2: {life_pct_p2:6.2f}%", end='', flush=True)
            
            # Draw overlay
            overlay = full_img.copy()
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 2.0
            thickness = 4
            
            # Player 1 text (above their health bar)
            p1_text = f"P1: {life_pct_p1:.1f}%"
            p1_pos = (p1_x + 100, y - 30)
            # Black outline
            cv2.putText(overlay, p1_text, p1_pos, font, font_scale, (0, 0, 0), thickness + 3)
            # Yellow text
            cv2.putText(overlay, p1_text, p1_pos, font, font_scale, (0, 255, 255), thickness)
            
            # Player 2 text (above their health bar)
            p2_text = f"P2: {life_pct_p2:.1f}%"
            p2_pos = (p2_x - 350, y - 30)
            # Black outline
            cv2.putText(overlay, p2_text, p2_pos, font, font_scale, (0, 0, 0), thickness + 3)
            # Yellow text
            cv2.putText(overlay, p2_text, p2_pos, font, font_scale, (0, 255, 255), thickness)
            
            # Optional: highlight health bars
            # cv2.rectangle(overlay, (p1_x, y-5), (p1_x + bar_len, y+5), (0, 255, 0), 2)
            # cv2.rectangle(overlay, (p2_x - bar_len, y-5), (p2_x, y+5), (0, 255, 0), 2)
            
            # Show the overlay
            cv2.imshow('Health Monitor', overlay)
            
            # Handle keyboard input
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('f'):
                if cv2.getWindowProperty('Health Monitor', cv2.WND_PROP_FULLSCREEN) == cv2.WINDOW_FULLSCREEN:
                    cv2.setWindowProperty('Health Monitor', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_NORMAL)
                else:
                    cv2.setWindowProperty('Health Monitor', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
            
            # Throttle to target FPS
            dt = time.perf_counter() - t0
            if dt < interval:
                time.sleep(interval - dt)
                
    except KeyboardInterrupt:
        print("\nStopped.")
    finally:
        cv2.destroyAllWindows()

if __name__ == "__main__":
    # Default configuration
    WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"
    PLAYER_1_X = 505      # P1 bar starts at 505
    PLAYER_2_X = 1421     # P2 bar starts at 1421 (but we'll calculate from 1021)
    BAR_LEN = 400
    Y = 155
    FPS = 30
    LOWER_BGR = np.array([0, 160, 190], dtype=np.uint8)
    UPPER_BGR = np.array([20, 180, 220], dtype=np.uint8)
    DROP_PER_PX = 0.25
    
    # Choose one:
    
    # 1. Monitor both players (terminal only) - YOUR WORKING VERSION
    # monitor_both_players(
    #     WINDOW_TITLE,
    #     PLAYER_1_X,
    #     PLAYER_2_X,
    #     BAR_LEN,
    #     Y,
    #     FPS,
    #     LOWER_BGR,
    #     UPPER_BGR,
    #     DROP_PER_PX
    # )
    
    # 2. Monitor both players with overlay
    monitor_both_players_with_overlay(
        WINDOW_TITLE,
        PLAYER_1_X,
        PLAYER_2_X,
        BAR_LEN,
        Y,
        FPS,
        LOWER_BGR,
        UPPER_BGR,
        DROP_PER_PX
    )

In [ ]:
import cv2
import numpy as np
from mss import mss
import win32gui

# STEP 1: Can we find your game window?
WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"

print(f"Looking for: '{WINDOW_TITLE}'")

hwnd = win32gui.FindWindow(None, WINDOW_TITLE)
if not hwnd:
    print("❌ Window NOT found!")
    print("\nLet me list windows with 'Bloody' or 'BizHawk' in the name:")
    
    def callback(hwnd, windows):
        text = win32gui.GetWindowText(hwnd)
        if text and ("Bloody" in text or "BizHawk" in text):
            print(f"  - '{text}'")
    
    win32gui.EnumWindows(callback, None)
    print("\nCopy the EXACT title and update WINDOW_TITLE in the code")
else:
    print("✅ Window found!")
    
    # STEP 2: Capture and show it
    rect = win32gui.GetClientRect(hwnd)
    left, top = win32gui.ClientToScreen(hwnd, (0, 0))
    
    monitor = {
        'left': left,
        'top': top,
        'width': rect[2],
        'height': rect[3]
    }
    
    sct = mss()
    screenshot = np.array(sct.grab(monitor))
    image = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
    
    cv2.imshow("Do you see your game?", image)
    print("\nDo you see your game in the window? Press any key...")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print("Good! Window capture works!")

In [ ]:
import cv2
import numpy as np
from mss import mss
import win32gui

WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"

hwnd = win32gui.FindWindow(None, WINDOW_TITLE)
rect = win32gui.GetClientRect(hwnd)
left, top = win32gui.ClientToScreen(hwnd, (0, 0))

monitor = {
    'left': left,
    'top': top,
    'width': rect[2],
    'height': rect[3]
}

sct = mss()

# Create background subtractor (detects movement)
bg_sub = cv2.createBackgroundSubtractorMOG2()

print("Keep the characters STILL for 3 seconds...")
for i in range(90):  # 3 seconds at 30fps
    screenshot = np.array(sct.grab(monitor))
    image = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
    bg_sub.apply(image)
    
print("NOW MOVE THE CHARACTERS!")
print("Press 'q' to quit\n")

while True:
    # Capture
    screenshot = np.array(sct.grab(monitor))
    image = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
    
    # Detect movement
    motion_mask = bg_sub.apply(image)
    
    # Show side by side
    display = np.hstack([
        cv2.resize(image, (640, 480)),  # Original
        cv2.cvtColor(cv2.resize(motion_mask, (640, 480)), cv2.COLOR_GRAY2BGR)  # Motion
    ])
    
    cv2.imshow("Left: Game | Right: Movement (white = moving)", display)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
print("Did you see white shapes on the right when characters moved?")

In [ ]:
import cv2
import numpy as np
from mss import mss
import win32gui

WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"

hwnd = win32gui.FindWindow(None, WINDOW_TITLE)
rect = win32gui.GetClientRect(hwnd)
left, top = win32gui.ClientToScreen(hwnd, (0, 0))

monitor = {
    'left': left,
    'top': top,
    'width': rect[2],
    'height': rect[3]
}

sct = mss()
bg_sub = cv2.createBackgroundSubtractorMOG2()

print("Learning background for 2 seconds...")
for i in range(60):
    screenshot = np.array(sct.grab(monitor))
    image = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
    bg_sub.apply(image)

print("Now detecting fighters! Press 'q' to quit\n")

while True:
    # Capture
    screenshot = np.array(sct.grab(monitor))
    image = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
    display = image.copy()
    
    # Get movement
    motion = bg_sub.apply(image)
    
    # Clean up noise
    kernel = np.ones((5,5), np.uint8)
    motion = cv2.morphologyEx(motion, cv2.MORPH_OPEN, kernel)
    motion = cv2.morphologyEx(motion, cv2.MORPH_CLOSE, kernel)
    
    # Find contours (shapes)
    contours, _ = cv2.findContours(motion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Find the 2 biggest moving objects
    valid_contours = []
    for c in contours:
        area = cv2.contourArea(c)
        if area > 500:  # Minimum size
            x, y, w, h = cv2.boundingRect(c)
            valid_contours.append((area, x, y, w, h))
    
    # Sort by size and get biggest 2
    valid_contours.sort(reverse=True)
    
    # Draw boxes around fighters
    for i, (area, x, y, w, h) in enumerate(valid_contours[:2]):
        center_x = x + w//2
        center_y = y + h//2
        
        color = (0, 255, 0) if i == 0 else (0, 0, 255)  # Green for P1, Red for P2
        label = "P1" if center_x < monitor['width']//2 else "P2"  # Left = P1, Right = P2
        
        # Draw box and center point
        cv2.rectangle(display, (x, y), (x+w, y+h), color, 2)
        cv2.circle(display, (center_x, center_y), 5, color, -1)
        cv2.putText(display, f"{label}: ({center_x}, {center_y})", 
                   (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    # Show result
    cv2.imshow("Fighter Detection", cv2.resize(display, (800, 600)))
    
    # Print coordinates
    if len(valid_contours) >= 2:
        p1_x = valid_contours[0][1] + valid_contours[0][3]//2
        p1_y = valid_contours[0][2] + valid_contours[0][4]//2
        p2_x = valid_contours[1][1] + valid_contours[1][3]//2
        p2_y = valid_contours[1][2] + valid_contours[1][4]//2
        
        # Determine which is which based on X position
        if p1_x > p2_x:
            p1_x, p2_x = p2_x, p1_x
            p1_y, p2_y = p2_y, p1_y
            
        print(f"\rP1: ({p1_x}, {p1_y}) | P2: ({p2_x}, {p2_y}) | Distance: {abs(p2_x - p1_x)}px", end='')
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from mss import mss
import win32gui

WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"

hwnd = win32gui.FindWindow(None, WINDOW_TITLE)
rect = win32gui.GetClientRect(hwnd)
left, top = win32gui.ClientToScreen(hwnd, (0, 0))

# CROP THE GAME AREA - EXCLUDE UI
full_height = rect[3]
game_area_top = 60     # Skip top UI (health bars)
game_area_bottom = full_height - 100  # Skip bottom UI (color bar, etc)

monitor = {
    'left': left,
    'top': top + game_area_top,
    'width': rect[2],
    'height': game_area_bottom - game_area_top
}

sct = mss()
bg_sub = cv2.createBackgroundSubtractorMOG2(detectShadows=False, varThreshold=16)

print("Learning background for 2 seconds...")
for i in range(60):
    screenshot = np.array(sct.grab(monitor))
    image = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
    bg_sub.apply(image)

print("Detecting fighters! Press 'q' to quit\n")

# Simple position tracking
last_p1 = None
last_p2 = None

while True:
    screenshot = np.array(sct.grab(monitor))
    image = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
    display = image.copy()
    
    # Get movement
    motion = bg_sub.apply(image)
    
    # Clean up
    kernel = np.ones((7,7), np.uint8)
    motion = cv2.morphologyEx(motion, cv2.MORPH_OPEN, kernel)
    motion = cv2.morphologyEx(motion, cv2.MORPH_CLOSE, kernel)
    
    # Find contours
    contours, _ = cv2.findContours(motion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Get two largest contours
    valid = []
    for c in contours:
        area = cv2.contourArea(c)
        if area > 800:  # Minimum size
            x, y, w, h = cv2.boundingRect(c)
            center_x = x + w//2
            center_y = y + h//2 + game_area_top  # Adjust for crop
            valid.append((area, center_x, center_y, x, y, w, h))
    
    # Sort by area (largest first)
    valid.sort(reverse=True)
    
    # Take top 2 and sort by X position
    fighters = valid[:2]
    fighters.sort(key=lambda f: f[1])  # Sort by x position
    
    # Draw
    if len(fighters) >= 1:
        _, x, y, bx, by, w, h = fighters[0]
        cv2.rectangle(display, (bx, by), (bx+w, by+h), (0, 255, 0), 2)
        cv2.circle(display, (x-left, y-top-game_area_top), 5, (0, 255, 0), -1)
        cv2.putText(display, f"P1: ({x}, {y})", (bx, by-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        last_p1 = (x, y)
        
    if len(fighters) >= 2:
        _, x, y, bx, by, w, h = fighters[1]
        cv2.rectangle(display, (bx, by), (bx+w, by+h), (0, 0, 255), 2)
        cv2.circle(display, (x-left, y-top-game_area_top), 5, (0, 0, 255), -1)
        cv2.putText(display, f"P2: ({x}, {y})", (bx, by-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        last_p2 = (x, y)
    
    # Show
    cv2.imshow("Fighter Detection (Cropped Area)", display)
    
    if last_p1 and last_p2:
        print(f"\rP1: {last_p1} | P2: {last_p2} | Distance: {abs(last_p1[0] - last_p2[0])}px", end='')
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from mss import mss
import win32gui
from ultralytics import YOLO

# First install: pip install ultralytics

WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"

# Load YOLO model (will download automatically first time)
model = YOLO('yolov8n.pt')  # nano model for speed

hwnd = win32gui.FindWindow(None, WINDOW_TITLE)
rect = win32gui.GetClientRect(hwnd)
left, top = win32gui.ClientToScreen(hwnd, (0, 0))

monitor = {
    'left': left,
    'top': top,
    'width': rect[2],
    'height': rect[3]
}

sct = mss()

print("Starting YOLO detection... Press 'q' to quit\n")

while True:
    # Capture
    screenshot = np.array(sct.grab(monitor))
    image = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
    
    # Run YOLO
    results = model(image, classes=[0], conf=0.3)  # class 0 = person
    
    # Process results
    fighters = []
    for r in results:
        if r.boxes is not None:
            for box in r.boxes:
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)
                conf = box.conf[0].item()
                
                fighters.append({
                    'center': (center_x, center_y),
                    'box': (int(x1), int(y1), int(x2), int(y2)),
                    'conf': conf
                })
    
    # Sort by x position
    fighters.sort(key=lambda f: f['center'][0])
    
    # Draw detections
    display = image.copy()
    if len(fighters) >= 1:
        f = fighters[0]
        cv2.rectangle(display, f['box'][:2], f['box'][2:], (0, 255, 0), 2)
        cv2.circle(display, f['center'], 5, (0, 255, 0), -1)
        cv2.putText(display, f"P1: {f['center']} [{f['conf']:.2f}]", 
                   (f['box'][0], f['box'][1]-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    if len(fighters) >= 2:
        f = fighters[1]
        cv2.rectangle(display, f['box'][:2], f['box'][2:], (0, 0, 255), 2)
        cv2.circle(display, f['center'], 5, (0, 0, 255), -1)
        cv2.putText(display, f"P2: {f['center']} [{f['conf']:.2f}]", 
                   (f['box'][0], f['box'][1]-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    cv2.imshow("YOLO Fighter Detection", cv2.resize(display, (800, 600)))
    
    if len(fighters) >= 2:
        p1, p2 = fighters[0]['center'], fighters[1]['center']
        print(f"\rP1: {p1} | P2: {p2} | Distance: {abs(p1[0] - p2[0])}px", end='')
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

In [6]:
import cv2
import numpy as np
from mss import mss
import win32gui
import time
from ultralytics import YOLO

class CompleteGameMonitor:
    def __init__(self, window_title):
        self.window_title = window_title
        self.sct = mss()
        
        # YOLO for fighter detection
        print("Loading YOLO model...")
        self.model = YOLO('yolov8n.pt')  # Will download first time
        
        # Health bar parameters (from your working code)
        self.health_params = {
            'p1_x': 505,
            'p2_x': 1421,
            'bar_len': 400,
            'y': 155,
            'lower_bgr': np.array([0, 160, 190], dtype=np.uint8),
            'upper_bgr': np.array([20, 180, 220], dtype=np.uint8),
            'drop_per_px': 0.25
        }
        
        # Get window handle
        self.hwnd = win32gui.FindWindow(None, self.window_title)
        if not self.hwnd:
            raise RuntimeError(f"Window not found: {self.window_title}")
            
        # Window dimensions
        rect = win32gui.GetClientRect(self.hwnd)
        self.left, self.top = win32gui.ClientToScreen(self.hwnd, (0, 0))
        self.width = rect[2]
        self.height = rect[3]
        
        # Position tracking for stability
        self.last_p1_pos = None
        self.last_p2_pos = None
        
    def detect_health(self):
        """Detect health bars using your existing method."""
        # ROIs for health bars
        roi_p1 = {
            'left': self.left + self.health_params['p1_x'],
            'top': self.top + self.health_params['y'],
            'width': self.health_params['bar_len'],
            'height': 1,
        }
        
        roi_p2 = {
            'left': self.left + (self.health_params['p2_x'] - self.health_params['bar_len']),
            'top': self.top + self.health_params['y'],
            'width': self.health_params['bar_len'],
            'height': 1,
        }
        
        # Capture health bars
        raw_p1 = self.sct.grab(roi_p1)
        strip_p1 = np.array(raw_p1)[:, :, :3]
        b1, g1, r1 = strip_p1[0].T
        
        raw_p2 = self.sct.grab(roi_p2)
        strip_p2 = np.array(raw_p2)[:, :, :3]
        b2, g2, r2 = strip_p2[0].T
        
        # Process Player 1 (left to right)
        mask_p1 = (
            (r1 >= self.health_params['lower_bgr'][2]) & 
            (r1 <= self.health_params['upper_bgr'][2]) &
            (g1 >= self.health_params['lower_bgr'][1]) & 
            (g1 <= self.health_params['upper_bgr'][1]) &
            (b1 >= self.health_params['lower_bgr'][0]) & 
            (b1 <= self.health_params['upper_bgr'][0])
        )
        non_yellow_p1 = np.nonzero(~mask_p1)[0]
        last_idx_p1 = non_yellow_p1.max() if non_yellow_p1.size else -1
        drop_pixels_p1 = max(0, last_idx_p1 + 1)
        life_pct_p1 = 100.0 - (drop_pixels_p1 * self.health_params['drop_per_px'])
        life_pct_p1 = np.clip(life_pct_p1, 0.0, 100.0)
        
        # Process Player 2 (right to left)
        mask_p2 = (
            (r2 >= self.health_params['lower_bgr'][2]) & 
            (r2 <= self.health_params['upper_bgr'][2]) &
            (g2 >= self.health_params['lower_bgr'][1]) & 
            (g2 <= self.health_params['upper_bgr'][1]) &
            (b2 >= self.health_params['lower_bgr'][0]) & 
            (b2 <= self.health_params['upper_bgr'][0])
        )
        non_yellow_p2 = np.nonzero(~mask_p2)[0]
        last_idx_p2 = non_yellow_p2.min() if non_yellow_p2.size else self.health_params['bar_len']
        drop_pixels_p2 = self.health_params['bar_len'] - last_idx_p2
        life_pct_p2 = 100.0 - (drop_pixels_p2 * self.health_params['drop_per_px'])
        life_pct_p2 = np.clip(life_pct_p2, 0.0, 100.0)
        
        return life_pct_p1, life_pct_p2
    
    def detect_fighters(self, image):
        """Detect fighter positions using YOLO."""
        # Run YOLO detection
        results = self.model(image, classes=[0], conf=0.3, verbose=False)  # class 0 = person
        
        fighters = []
        for r in results:
            if r.boxes is not None:
                for box in r.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    center_x = int((x1 + x2) / 2)
                    center_y = int((y1 + y2) / 2)
                    conf = box.conf[0].item()
                    
                    # Skip if too high (might be UI elements)
                    if center_y > 100:  # Below health bars
                        fighters.append({
                            'center': (center_x, center_y),
                            'box': (int(x1), int(y1), int(x2), int(y2)),
                            'conf': conf
                        })
        
        # Sort by x position to identify P1 (left) and P2 (right)
        fighters.sort(key=lambda f: f['center'][0])
        
        # Assign to players
        p1_pos = None
        p2_pos = None
        
        if len(fighters) >= 2:
            p1_pos = fighters[0]['center']
            p2_pos = fighters[1]['center']
        elif len(fighters) == 1:
            # Use last known positions to determine which player
            pos = fighters[0]['center']
            if self.last_p1_pos and self.last_p2_pos:
                dist_to_p1 = abs(pos[0] - self.last_p1_pos[0])
                dist_to_p2 = abs(pos[0] - self.last_p2_pos[0])
                if dist_to_p1 < dist_to_p2:
                    p1_pos = pos
                else:
                    p2_pos = pos
            else:
                # Guess based on position
                if pos[0] < self.width // 2:
                    p1_pos = pos
                else:
                    p2_pos = pos
        
        # Update last known positions
        if p1_pos:
            self.last_p1_pos = p1_pos
        if p2_pos:
            self.last_p2_pos = p2_pos
        
        return {
            'p1': p1_pos,
            'p2': p2_pos,
            'distance': abs(p1_pos[0] - p2_pos[0]) if p1_pos and p2_pos else None,
            'all_detections': fighters
        }
    
    def get_game_state(self):
        """Get complete game state including positions and health."""
        # Capture full screen
        monitor = {
            'left': self.left,
            'top': self.top,
            'width': self.width,
            'height': self.height
        }
        
        screenshot = np.array(self.sct.grab(monitor))
        image = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
        
        # Get fighter positions
        fighter_data = self.detect_fighters(image)
        
        # Get health
        p1_health, p2_health = self.detect_health()
        
        # Combine all data
        game_state = {
            'p1': {
                'position': fighter_data['p1'],
                'health': p1_health
            },
            'p2': {
                'position': fighter_data['p2'],
                'health': p2_health
            },
            'distance': fighter_data['distance'],
            'frame': image,
            'detections': fighter_data['all_detections']
        }
        
        return game_state
    
    def visualize_state(self, show_window=True):
        """Run the monitor with visualization."""
        cv2.namedWindow('Game Monitor', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Game Monitor', self.width // 2, self.height // 2)
        
        fps = 30
        interval = 1.0 / fps
        
        print("Game monitor running. Press 'q' to quit\n")
        
        try:
            while True:
                t0 = time.perf_counter()
                
                # Get game state
                state = self.get_game_state()
                
                # Draw visualization
                display = state['frame'].copy()
                
                # Draw fighter positions
                if state['p1']['position']:
                    x, y = state['p1']['position']
                    cv2.circle(display, (x, y), 10, (0, 255, 0), -1)
                    cv2.putText(display, f"P1: {state['p1']['health']:.1f}%", 
                               (x-50, y-30), cv2.FONT_HERSHEY_SIMPLEX, 
                               0.7, (0, 255, 0), 2)
                
                if state['p2']['position']:
                    x, y = state['p2']['position']
                    cv2.circle(display, (x, y), 10, (0, 0, 255), -1)
                    cv2.putText(display, f"P2: {state['p2']['health']:.1f}%", 
                               (x-50, y-30), cv2.FONT_HERSHEY_SIMPLEX, 
                               0.7, (0, 0, 255), 2)
                
                # Draw health bars with outlines
                # P1 health bar
                cv2.rectangle(display, 
                             (self.health_params['p1_x'], self.health_params['y'] - 5),
                             (self.health_params['p1_x'] + self.health_params['bar_len'], 
                              self.health_params['y'] + 5),
                             (0, 255, 0), 2)
                
                # P2 health bar
                cv2.rectangle(display, 
                             (self.health_params['p2_x'] - self.health_params['bar_len'], 
                              self.health_params['y'] - 5),
                             (self.health_params['p2_x'], self.health_params['y'] + 5),
                             (0, 0, 255), 2)
                
                # Draw distance
                if state['distance']:
                    cv2.putText(display, f"Distance: {state['distance']}px", 
                               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                               1, (255, 255, 0), 2)
                
                # Draw all YOLO detections
                for det in state['detections']:
                    x1, y1, x2, y2 = det['box']
                    cv2.rectangle(display, (x1, y1), (x2, y2), (128, 128, 128), 1)
                    cv2.putText(display, f"{det['conf']:.2f}", 
                               (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 
                               0.4, (128, 128, 128), 1)
                
                # Print to console
                p1_str = f"P1: pos={state['p1']['position']}, hp={state['p1']['health']:.1f}%"
                p2_str = f"P2: pos={state['p2']['position']}, hp={state['p2']['health']:.1f}%"
                dist_str = f"Dist: {state['distance']}px" if state['distance'] else "Dist: ---"
                print(f"\r{p1_str} | {p2_str} | {dist_str}", end='', flush=True)
                
                # Show window
                if show_window:
                    cv2.imshow('Game Monitor', display)
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
                
                # Maintain FPS
                dt = time.perf_counter() - t0
                if dt < interval:
                    time.sleep(interval - dt)
                    
        except KeyboardInterrupt:
            print("\nStopped.")
        finally:
            cv2.destroyAllWindows()


# Usage for RL environment
class GameEnvironment:
    def __init__(self, window_title):
        self.monitor = CompleteGameMonitor(window_title)
        
    def get_observation(self):
        """Get current game state for RL agent."""
        state = self.monitor.get_game_state()
        
        # Format for RL (example)
        obs = {
            'p1_x': state['p1']['position'][0] if state['p1']['position'] else 0,
            'p1_y': state['p1']['position'][1] if state['p1']['position'] else 0,
            'p1_health': state['p1']['health'],
            'p2_x': state['p2']['position'][0] if state['p2']['position'] else 0,
            'p2_y': state['p2']['position'][1] if state['p2']['position'] else 0,
            'p2_health': state['p2']['health'],
            'distance': state['distance'] if state['distance'] else 0
        }
        
        return obs


if __name__ == "__main__":
    WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"
    
    # Run with visualization
    monitor = CompleteGameMonitor(WINDOW_TITLE)
    monitor.visualize_state()
    
    # Or use for RL
    # env = GameEnvironment(WINDOW_TITLE)
    # while True:
    #     obs = env.get_observation()
    #     print(obs)
    #     time.sleep(0.033)  # 30 FPS

Loading YOLO model...
Game monitor running. Press 'q' to quit

P1: pos=(814, 543), hp=48.0% | P2: pos=(1121, 576), hp=57.0% | Dist: 307px
Stopped.


In [ ]:
import cv2
import numpy as np
from mss import mss
import win32gui
import time
from ultralytics import YOLO

class CompleteGameMonitor:
    def __init__(self, window_title):
        self.window_title = window_title
        self.sct = mss()
        
        # YOLO for fighter detection
        print("Loading YOLO model...")
        self.model = YOLO('yolov8n.pt')  # Will download first time
        
        # Health bar parameters (from your working code)
        self.health_params = {
            'p1_x': 505,
            'p2_x': 1421,
            'bar_len': 400,
            'y': 155,
            'lower_bgr': np.array([0, 160, 190], dtype=np.uint8),
            'upper_bgr': np.array([20, 180, 220], dtype=np.uint8),
            'drop_per_px': 0.25
        }
        
        # Get window handle
        self.hwnd = win32gui.FindWindow(None, self.window_title)
        if not self.hwnd:
            raise RuntimeError(f"Window not found: {self.window_title}")
            
        # Window dimensions
        rect = win32gui.GetClientRect(self.hwnd)
        self.left, self.top = win32gui.ClientToScreen(self.hwnd, (0, 0))
        self.width = rect[2]
        self.height = rect[3]
        
        # Position tracking for stability
        self.last_p1_pos = None
        self.last_p2_pos = None
        
    def detect_health(self):
        """Detect health bars using your existing method."""
        # ROIs for health bars
        roi_p1 = {
            'left': self.left + self.health_params['p1_x'],
            'top': self.top + self.health_params['y'],
            'width': self.health_params['bar_len'],
            'height': 1,
        }
        
        roi_p2 = {
            'left': self.left + (self.health_params['p2_x'] - self.health_params['bar_len']),
            'top': self.top + self.health_params['y'],
            'width': self.health_params['bar_len'],
            'height': 1,
        }
        
        # Capture health bars
        raw_p1 = self.sct.grab(roi_p1)
        strip_p1 = np.array(raw_p1)[:, :, :3]
        b1, g1, r1 = strip_p1[0].T
        
        raw_p2 = self.sct.grab(roi_p2)
        strip_p2 = np.array(raw_p2)[:, :, :3]
        b2, g2, r2 = strip_p2[0].T
        
        # Process Player 1 (left to right)
        mask_p1 = (
            (r1 >= self.health_params['lower_bgr'][2]) & 
            (r1 <= self.health_params['upper_bgr'][2]) &
            (g1 >= self.health_params['lower_bgr'][1]) & 
            (g1 <= self.health_params['upper_bgr'][1]) &
            (b1 >= self.health_params['lower_bgr'][0]) & 
            (b1 <= self.health_params['upper_bgr'][0])
        )
        non_yellow_p1 = np.nonzero(~mask_p1)[0]
        last_idx_p1 = non_yellow_p1.max() if non_yellow_p1.size else -1
        drop_pixels_p1 = max(0, last_idx_p1 + 1)
        life_pct_p1 = 100.0 - (drop_pixels_p1 * self.health_params['drop_per_px'])
        life_pct_p1 = np.clip(life_pct_p1, 0.0, 100.0)
        
        # Process Player 2 (right to left)
        mask_p2 = (
            (r2 >= self.health_params['lower_bgr'][2]) & 
            (r2 <= self.health_params['upper_bgr'][2]) &
            (g2 >= self.health_params['lower_bgr'][1]) & 
            (g2 <= self.health_params['upper_bgr'][1]) &
            (b2 >= self.health_params['lower_bgr'][0]) & 
            (b2 <= self.health_params['upper_bgr'][0])
        )
        non_yellow_p2 = np.nonzero(~mask_p2)[0]
        last_idx_p2 = non_yellow_p2.min() if non_yellow_p2.size else self.health_params['bar_len']
        drop_pixels_p2 = self.health_params['bar_len'] - last_idx_p2
        life_pct_p2 = 100.0 - (drop_pixels_p2 * self.health_params['drop_per_px'])
        life_pct_p2 = np.clip(life_pct_p2, 0.0, 100.0)
        
        return life_pct_p1, life_pct_p2
    
    def detect_fighters(self, image):
        """Detect fighter positions using YOLO."""
        # Run YOLO detection
        results = self.model(image, classes=[0], conf=0.3, verbose=False)  # class 0 = person
        
        fighters = []
        for r in results:
            if r.boxes is not None:
                for box in r.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    center_x = int((x1 + x2) / 2)
                    center_y = int((y1 + y2) / 2)
                    conf = box.conf[0].item()
                    
                    # Skip if too high (might be UI elements)
                    if center_y > 100:  # Below health bars
                        fighters.append({
                            'center': (center_x, center_y),
                            'box': (int(x1), int(y1), int(x2), int(y2)),
                            'conf': conf
                        })
        
        # Sort by x position to identify P1 (left) and P2 (right)
        fighters.sort(key=lambda f: f['center'][0])
        
        # Assign to players
        p1_pos = None
        p2_pos = None
        
        if len(fighters) >= 2:
            p1_pos = fighters[0]['center']
            p2_pos = fighters[1]['center']
        elif len(fighters) == 1:
            # Use last known positions to determine which player
            pos = fighters[0]['center']
            if self.last_p1_pos and self.last_p2_pos:
                dist_to_p1 = abs(pos[0] - self.last_p1_pos[0])
                dist_to_p2 = abs(pos[0] - self.last_p2_pos[0])
                if dist_to_p1 < dist_to_p2:
                    p1_pos = pos
                else:
                    p2_pos = pos
            else:
                # Guess based on position
                if pos[0] < self.width // 2:
                    p1_pos = pos
                else:
                    p2_pos = pos
        
        # Update last known positions
        if p1_pos:
            self.last_p1_pos = p1_pos
        if p2_pos:
            self.last_p2_pos = p2_pos
        
        return {
            'p1': p1_pos,
            'p2': p2_pos,
            'distance': abs(p1_pos[0] - p2_pos[0]) if p1_pos and p2_pos else None,
            'all_detections': fighters
        }
    
    def get_game_state(self):
        """Get complete game state including positions and health."""
        # Time each component
        t_start = time.perf_counter()
        
        # Capture full screen
        monitor = {
            'left': self.left,
            'top': self.top,
            'width': self.width,
            'height': self.height
        }
        
        screenshot = np.array(self.sct.grab(monitor))
        image = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
        t_capture = time.perf_counter()
        
        # Get fighter positions
        fighter_data = self.detect_fighters(image)
        t_detection = time.perf_counter()
        
        # Get health
        p1_health, p2_health = self.detect_health()
        t_health = time.perf_counter()
        
        # Combine all data
        game_state = {
            'p1': {
                'position': fighter_data['p1'],
                'health': p1_health
            },
            'p2': {
                'position': fighter_data['p2'],
                'health': p2_health
            },
            'distance': fighter_data['distance'],
            'frame': image,
            'detections': fighter_data['all_detections'],
            'timing': {
                'capture_ms': (t_capture - t_start) * 1000,
                'detection_ms': (t_detection - t_capture) * 1000,
                'health_ms': (t_health - t_detection) * 1000,
                'total_ms': (t_health - t_start) * 1000
            }
        }
        
        return game_state
    
    def visualize_state(self, show_window=True):
        """Run the monitor with visualization."""
        cv2.namedWindow('Game Monitor', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Game Monitor', self.width // 2, self.height // 2)
        
        fps = 30
        interval = 1.0 / fps
        
        # Timing statistics
        frame_times = []
        detection_times = []
        health_times = []
        
        print("Game monitor running. Press 'q' to quit")
        print("Press 't' to show/hide timing info\n")
        
        show_timing = True
        
        try:
            while True:
                t0 = time.perf_counter()
                
                # Get game state with timing
                t_start = time.perf_counter()
                state = self.get_game_state()
                total_time = (time.perf_counter() - t_start) * 1000  # ms
                frame_times.append(total_time)
                
                # Draw visualization
                display = state['frame'].copy()
                
                # Draw fighter positions with CENTER MARKERS
                if state['p1']['position']:
                    x, y = state['p1']['position']  # This IS the center!
                    # Draw crosshair at center
                    cv2.line(display, (x-15, y), (x+15, y), (0, 255, 0), 2)
                    cv2.line(display, (x, y-15), (x, y+15), (0, 255, 0), 2)
                    cv2.circle(display, (x, y), 5, (0, 255, 0), -1)
                    cv2.putText(display, f"P1: {state['p1']['health']:.1f}%", 
                               (x-50, y-30), cv2.FONT_HERSHEY_SIMPLEX, 
                               0.7, (0, 255, 0), 2)
                
                if state['p2']['position']:
                    x, y = state['p2']['position']  # This IS the center!
                    # Draw crosshair at center
                    cv2.line(display, (x-15, y), (x+15, y), (0, 0, 255), 2)
                    cv2.line(display, (x, y-15), (x, y+15), (0, 0, 255), 2)
                    cv2.circle(display, (x, y), 5, (0, 0, 255), -1)
                    cv2.putText(display, f"P2: {state['p2']['health']:.1f}%", 
                               (x-50, y-30), cv2.FONT_HERSHEY_SIMPLEX, 
                               0.7, (0, 0, 255), 2)
                
                # Draw health bars with outlines
                # P1 health bar
                cv2.rectangle(display, 
                             (self.health_params['p1_x'], self.health_params['y'] - 5),
                             (self.health_params['p1_x'] + self.health_params['bar_len'], 
                              self.health_params['y'] + 5),
                             (0, 255, 0), 2)
                
                # P2 health bar
                cv2.rectangle(display, 
                             (self.health_params['p2_x'] - self.health_params['bar_len'], 
                              self.health_params['y'] - 5),
                             (self.health_params['p2_x'], self.health_params['y'] + 5),
                             (0, 0, 255), 2)
                
                # Draw distance
                if state['distance']:
                    cv2.putText(display, f"Distance: {state['distance']}px", 
                               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                               1, (255, 255, 0), 2)
                
                # Draw all YOLO detections
                for det in state['detections']:
                    x1, y1, x2, y2 = det['box']
                    cv2.rectangle(display, (x1, y1), (x2, y2), (128, 128, 128), 1)
                    cv2.putText(display, f"{det['conf']:.2f}", 
                               (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 
                               0.4, (128, 128, 128), 1)
                
                # Show timing info
                if show_timing and len(frame_times) > 10:
                    avg_time = np.mean(frame_times[-30:])
                    actual_fps = 1000.0 / avg_time if avg_time > 0 else 0
                    cv2.putText(display, f"Processing: {avg_time:.1f}ms ({actual_fps:.1f} FPS)", 
                               (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 
                               0.7, (0, 255, 255), 2)
                
                # Print to console with timing
                p1_str = f"P1: pos={state['p1']['position']}, hp={state['p1']['health']:.1f}%"
                p2_str = f"P2: pos={state['p2']['position']}, hp={state['p2']['health']:.1f}%"
                dist_str = f"Dist: {state['distance']}px" if state['distance'] else "Dist: ---"
                time_str = f"Time: {total_time:.1f}ms"
                print(f"\r{p1_str} | {p2_str} | {dist_str} | {time_str}", end='', flush=True)
                
                # Show window
                if show_window:
                    cv2.imshow('Game Monitor', display)
                    key = cv2.waitKey(1) & 0xFF
                    if key == ord('q'):
                        break
                    elif key == ord('t'):
                        show_timing = not show_timing
                
                # Maintain FPS
                dt = time.perf_counter() - t0
                if dt < interval:
                    time.sleep(interval - dt)
                    
        except KeyboardInterrupt:
            print("\nStopped.")
        finally:
            cv2.destroyAllWindows()


# Usage for RL environment
class GameEnvironment:
    def __init__(self, window_title):
        self.monitor = CompleteGameMonitor(window_title)
        
    def get_observation(self):
        """Get current game state for RL agent."""
        state = self.monitor.get_game_state()
        
        # Format for RL (example)
        obs = {
            'p1_x': state['p1']['position'][0] if state['p1']['position'] else 0,
            'p1_y': state['p1']['position'][1] if state['p1']['position'] else 0,
            'p1_health': state['p1']['health'],
            'p2_x': state['p2']['position'][0] if state['p2']['position'] else 0,
            'p2_y': state['p2']['position'][1] if state['p2']['position'] else 0,
            'p2_health': state['p2']['health'],
            'distance': state['distance'] if state['distance'] else 0
        }
        
        return obs


    def benchmark_performance(self, num_frames=100):
        """Test performance over multiple frames."""
        print(f"Benchmarking performance over {num_frames} frames...")
        
        timings = {
            'capture': [],
            'detection': [],
            'health': [],
            'total': []
        }
        
        for i in range(num_frames):
            state = self.get_game_state()
            timings['capture'].append(state['timing']['capture_ms'])
            timings['detection'].append(state['timing']['detection_ms'])
            timings['health'].append(state['timing']['health_ms'])
            timings['total'].append(state['timing']['total_ms'])
            
            if i % 10 == 0:
                print(f"\rProgress: {i}/{num_frames}", end='')
        
        print("\n\nPerformance Results:")
        print("-" * 40)
        print(f"Screen Capture:  {np.mean(timings['capture']):.2f} ms (±{np.std(timings['capture']):.2f})")
        print(f"YOLO Detection:  {np.mean(timings['detection']):.2f} ms (±{np.std(timings['detection']):.2f})")
        print(f"Health Detection: {np.mean(timings['health']):.2f} ms (±{np.std(timings['health']):.2f})")
        print(f"TOTAL:           {np.mean(timings['total']):.2f} ms (±{np.std(timings['total']):.2f})")
        print(f"Average FPS:     {1000.0 / np.mean(timings['total']):.1f}")
        print("-" * 40)


if __name__ == "__main__":
    WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"
    
    # Run benchmark first
    monitor = CompleteGameMonitor(WINDOW_TITLE)
    monitor.benchmark_performance(100)
    
    print("\nPress Enter to start visual monitor...")
    input()
    
    # Run with visualization
    monitor.visualize_state()
    
    # Or use for RL
    # env = GameEnvironment(WINDOW_TITLE)
    # while True:
    #     obs = env.get_observation()
    #     print(obs)
    #     time.sleep(0.033)  # 30 FPS

In [5]:
import cv2
import numpy as np
from mss import mss
import win32gui
import time
from ultralytics import YOLO
import torch

class OptimizedGameMonitor:
    def __init__(self, window_title, use_gpu=True, detection_interval=3):
        self.window_title = window_title
        self.sct = mss()
        self.detection_interval = detection_interval  # Run YOLO every N frames
        self.frame_count = 0
        
        # YOLO optimization
        print("Loading YOLO model...")
        self.model = YOLO('yolov8n.pt')
        
        # Force GPU if available
        if use_gpu and torch.cuda.is_available():
            self.model.to('cuda')
            print("✅ Using GPU for YOLO")
        else:
            print("⚠️ Using CPU for YOLO (slower)")
        
        # Health bar parameters
        self.health_params = {
            'p1_x': 505,
            'p2_x': 1421,
            'bar_len': 400,
            'y': 155,
            'lower_bgr': np.array([0, 160, 190], dtype=np.uint8),
            'upper_bgr': np.array([20, 180, 220], dtype=np.uint8),
            'drop_per_px': 0.25
        }
        
        # Window setup
        self.hwnd = win32gui.FindWindow(None, self.window_title)
        if not self.hwnd:
            raise RuntimeError(f"Window not found: {self.window_title}")
            
        rect = win32gui.GetClientRect(self.hwnd)
        self.left, self.top = win32gui.ClientToScreen(self.hwnd, (0, 0))
        self.width = rect[2]
        self.height = rect[3]
        
        # Pre-define monitor region (faster than creating dict each time)
        self.monitor = {
            'left': self.left,
            'top': self.top,
            'width': self.width,
            'height': self.height
        }
        
        # Pre-define health ROIs
        self.roi_p1 = {
            'left': self.left + self.health_params['p1_x'],
            'top': self.top + self.health_params['y'],
            'width': self.health_params['bar_len'],
            'height': 1,
        }
        
        self.roi_p2 = {
            'left': self.left + (self.health_params['p2_x'] - self.health_params['bar_len']),
            'top': self.top + self.health_params['y'],
            'width': self.health_params['bar_len'],
            'height': 1,
        }
        
        # Cached positions for interpolation
        self.last_p1_pos = None
        self.last_p2_pos = None
        self.cached_fighters = None
        
    def detect_health_optimized(self):
        """Optimized health detection - capture both bars in one grab."""
        # Single capture for both health bars
        health_region = {
            'left': self.left,
            'top': self.top + self.health_params['y'],
            'width': self.width,
            'height': 1
        }
        
        health_strip = np.array(self.sct.grab(health_region))[:, :, :3]
        
        # Extract P1 health (direct slice)
        p1_start = self.health_params['p1_x']
        p1_end = p1_start + self.health_params['bar_len']
        strip_p1 = health_strip[0, p1_start:p1_end]
        
        # Extract P2 health (direct slice)
        p2_start = self.health_params['p2_x'] - self.health_params['bar_len']
        p2_end = self.health_params['p2_x']
        strip_p2 = health_strip[0, p2_start:p2_end]
        
        # Vectorized color matching for P1
        mask_p1 = (
            (strip_p1[:, 2] >= self.health_params['lower_bgr'][2]) & 
            (strip_p1[:, 2] <= self.health_params['upper_bgr'][2]) &
            (strip_p1[:, 1] >= self.health_params['lower_bgr'][1]) & 
            (strip_p1[:, 1] <= self.health_params['upper_bgr'][1]) &
            (strip_p1[:, 0] >= self.health_params['lower_bgr'][0]) & 
            (strip_p1[:, 0] <= self.health_params['upper_bgr'][0])
        )
        
        # Vectorized color matching for P2
        mask_p2 = (
            (strip_p2[:, 2] >= self.health_params['lower_bgr'][2]) & 
            (strip_p2[:, 2] <= self.health_params['upper_bgr'][2]) &
            (strip_p2[:, 1] >= self.health_params['lower_bgr'][1]) & 
            (strip_p2[:, 1] <= self.health_params['upper_bgr'][1]) &
            (strip_p2[:, 0] >= self.health_params['lower_bgr'][0]) & 
            (strip_p2[:, 0] <= self.health_params['upper_bgr'][0])
        )
        
        # Calculate health
        non_yellow_p1 = np.where(~mask_p1)[0]
        last_idx_p1 = non_yellow_p1[-1] if len(non_yellow_p1) > 0 else -1
        life_pct_p1 = 100.0 - (max(0, last_idx_p1 + 1) * self.health_params['drop_per_px'])
        
        non_yellow_p2 = np.where(~mask_p2)[0]
        last_idx_p2 = non_yellow_p2[0] if len(non_yellow_p2) > 0 else self.health_params['bar_len']
        life_pct_p2 = 100.0 - ((self.health_params['bar_len'] - last_idx_p2) * self.health_params['drop_per_px'])
        
        return np.clip(life_pct_p1, 0, 100), np.clip(life_pct_p2, 0, 100)
    
    def detect_fighters_cached(self, image):
        """Run YOLO only every N frames, interpolate between."""
        self.frame_count += 1
        
        # Run detection only every N frames
        if self.frame_count % self.detection_interval == 0:
            # Resize image for faster YOLO (trade accuracy for speed)
            small_image = cv2.resize(image, (640, 480))
            
            # Run YOLO
            results = self.model(small_image, classes=[0], conf=0.3, verbose=False)
            
            fighters = []
            scale_x = image.shape[1] / 640
            scale_y = image.shape[0] / 480
            
            for r in results:
                if r.boxes is not None:
                    for box in r.boxes:
                        x1, y1, x2, y2 = box.xyxy[0].tolist()
                        # Scale back to original size
                        center_x = int((x1 + x2) / 2 * scale_x)
                        center_y = int((y1 + y2) / 2 * scale_y)
                        
                        if center_y > 100:
                            fighters.append({
                                'center': (center_x, center_y),
                                'conf': box.conf[0].item()
                            })
            
            # Sort and assign
            fighters.sort(key=lambda f: f['center'][0])
            
            if len(fighters) >= 2:
                self.last_p1_pos = fighters[0]['center']
                self.last_p2_pos = fighters[1]['center']
            elif len(fighters) == 1:
                pos = fighters[0]['center']
                if self.last_p1_pos and self.last_p2_pos:
                    if abs(pos[0] - self.last_p1_pos[0]) < abs(pos[0] - self.last_p2_pos[0]):
                        self.last_p1_pos = pos
                    else:
                        self.last_p2_pos = pos
            
            self.cached_fighters = {
                'p1': self.last_p1_pos,
                'p2': self.last_p2_pos,
                'distance': abs(self.last_p1_pos[0] - self.last_p2_pos[0]) if self.last_p1_pos and self.last_p2_pos else None
            }
        
        return self.cached_fighters or {'p1': None, 'p2': None, 'distance': None}
    
    def get_game_state_fast(self):
        """Optimized game state capture."""
        t_start = time.perf_counter()
        
        # Single screen capture
        screenshot = np.array(self.sct.grab(self.monitor))
        image = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
        t_capture = time.perf_counter()
        
        # Fighter detection (cached)
        fighter_data = self.detect_fighters_cached(image)
        t_detection = time.perf_counter()
        
        # Health detection (optimized)
        p1_health, p2_health = self.detect_health_optimized()
        t_health = time.perf_counter()
        
        return {
            'p1': {
                'position': fighter_data['p1'],
                'health': p1_health
            },
            'p2': {
                'position': fighter_data['p2'],
                'health': p2_health
            },
            'distance': fighter_data['distance'],
            'frame': image,
            'timing': {
                'capture_ms': (t_capture - t_start) * 1000,
                'detection_ms': (t_detection - t_capture) * 1000,
                'health_ms': (t_health - t_detection) * 1000,
                'total_ms': (t_health - t_start) * 1000
            }
        }
    
    def benchmark_optimized(self, num_frames=100):
        """Benchmark optimized performance."""
        print(f"Benchmarking optimized performance over {num_frames} frames...")
        print(f"YOLO runs every {self.detection_interval} frames")
        
        timings = {'capture': [], 'detection': [], 'health': [], 'total': []}
        
        for i in range(num_frames):
            state = self.get_game_state_fast()
            timings['capture'].append(state['timing']['capture_ms'])
            timings['detection'].append(state['timing']['detection_ms'])
            timings['health'].append(state['timing']['health_ms'])
            timings['total'].append(state['timing']['total_ms'])
            
            if i % 20 == 0:
                print(f"\rProgress: {i}/{num_frames}", end='')
        
        print("\n\nOptimized Performance Results:")
        print("-" * 40)
        print(f"Screen Capture:  {np.mean(timings['capture']):.2f} ms")
        print(f"YOLO Detection:  {np.mean(timings['detection']):.2f} ms")
        print(f"Health Detection: {np.mean(timings['health']):.2f} ms")
        print(f"TOTAL:           {np.mean(timings['total']):.2f} ms")
        print(f"Average FPS:     {1000.0 / np.mean(timings['total']):.1f}")
        print("-" * 40)


# Alternative: Motion-based detection (MUCH faster)
class MotionBasedMonitor:
    def __init__(self, window_title):
        self.window_title = window_title
        self.sct = mss()
        
        # Window setup
        self.hwnd = win32gui.FindWindow(None, self.window_title)
        rect = win32gui.GetClientRect(self.hwnd)
        self.left, self.top = win32gui.ClientToScreen(self.hwnd, (0, 0))
        self.width = rect[2]
        self.height = rect[3]
        
        # Motion detector
        self.bg_sub = cv2.createBackgroundSubtractorMOG2(
            detectShadows=False,
            varThreshold=16
        )
        
        # Same health params as before
        self.health_params = {
            'p1_x': 505,
            'p2_x': 1421,
            'bar_len': 400,
            'y': 155,
            'lower_bgr': np.array([0, 160, 190], dtype=np.uint8),
            'upper_bgr': np.array([20, 180, 220], dtype=np.uint8),
            'drop_per_px': 0.25
        }
        
    def detect_fighters_motion(self, image):
        """Ultra-fast motion-based detection."""
        # Crop to game area only
        game_area = image[100:self.height-100, :]
        
        # Get motion mask
        motion = self.bg_sub.apply(game_area)
        
        # Quick noise removal
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        motion = cv2.morphologyEx(motion, cv2.MORPH_OPEN, kernel)
        
        # Find contours
        contours, _ = cv2.findContours(motion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Quick filtering
        fighters = []
        for c in contours:
            area = cv2.contourArea(c)
            if 500 < area < 50000:
                M = cv2.moments(c)
                if M["m00"] > 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"]) + 100  # Add back crop offset
                    fighters.append((cx, cy, area))
        
        # Sort by x position
        fighters.sort(key=lambda f: f[0])
        
        p1_pos = None
        p2_pos = None
        
        if len(fighters) >= 2:
            p1_pos = (fighters[0][0], fighters[0][1])
            p2_pos = (fighters[1][0], fighters[1][1])
        
        return {
            'p1': p1_pos,
            'p2': p2_pos,
            'distance': abs(p1_pos[0] - p2_pos[0]) if p1_pos and p2_pos else None
        }
    
    def get_game_state_ultrafast(self):
        """Ultra-fast game state for 20+ FPS."""
        t_start = time.perf_counter()
        
        # Single capture
        monitor = {
            'left': self.left,
            'top': self.top,
            'width': self.width,
            'height': self.height
        }
        
        screenshot = np.array(self.sct.grab(monitor))
        image = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
        t_capture = time.perf_counter()
        
        # Motion detection
        fighters = self.detect_fighters_motion(image)
        t_detection = time.perf_counter()
        
        # Fast health detection (single strip)
        health_strip = image[self.health_params['y'], :]
        
        # P1 health
        p1_strip = health_strip[self.health_params['p1_x']:self.health_params['p1_x']+self.health_params['bar_len']]
        mask_p1 = ((p1_strip[:, 2] >= self.health_params['lower_bgr'][2]) & 
                   (p1_strip[:, 1] >= self.health_params['lower_bgr'][1]) & 
                   (p1_strip[:, 0] >= self.health_params['lower_bgr'][0]))
        p1_health = 100.0 - (np.argmin(mask_p1) * self.health_params['drop_per_px'] if not mask_p1.all() else 0)
        
        # P2 health
        p2_strip = health_strip[self.health_params['p2_x']-self.health_params['bar_len']:self.health_params['p2_x']]
        mask_p2 = ((p2_strip[:, 2] >= self.health_params['lower_bgr'][2]) & 
                   (p2_strip[:, 1] >= self.health_params['lower_bgr'][1]) & 
                   (p2_strip[:, 0] >= self.health_params['lower_bgr'][0]))
        p2_health = 100.0 - ((self.health_params['bar_len'] - np.argmax(~mask_p2)) * self.health_params['drop_per_px'] if not mask_p2.all() else 0)
        
        t_health = time.perf_counter()
        
        return {
            'p1': {'position': fighters['p1'], 'health': np.clip(p1_health, 0, 100)},
            'p2': {'position': fighters['p2'], 'health': np.clip(p2_health, 0, 100)},
            'distance': fighters['distance'],
            'timing': {
                'capture_ms': (t_capture - t_start) * 1000,
                'detection_ms': (t_detection - t_capture) * 1000,
                'health_ms': (t_health - t_detection) * 1000,
                'total_ms': (t_health - t_start) * 1000
            }
        }


if __name__ == "__main__":
    WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"
    
    print("Choose detection method:")
    print("1. Optimized YOLO (more accurate, ~10-15 FPS)")
    print("2. Motion Detection (less accurate, ~20-30 FPS)")
    
    choice = input("Enter 1 or 2: ")
    
    if choice == "1":
        monitor = OptimizedGameMonitor(WINDOW_TITLE, use_gpu=True, detection_interval=3)
        monitor.benchmark_optimized(100)
    else:
        monitor = MotionBasedMonitor(WINDOW_TITLE)
        print("Benchmarking motion-based detection...")
        
        times = []
        for i in range(100):
            state = monitor.get_game_state_ultrafast()
            times.append(state['timing']['total_ms'])
            if i % 20 == 0:
                print(f"\rProgress: {i}/100", end='')
        
        print(f"\n\nMotion Detection Perforrmance:")
        print(f"Average: {np.mean(times):.2f} ms")
        print(f"FPS: {1000.0 / np.mean(times):.1f}")

Choose detection method:
1. Optimized YOLO (more accurate, ~10-15 FPS)
2. Motion Detection (less accurate, ~20-30 FPS)
Loading YOLO model...
⚠️ Using CPU for YOLO (slower)
Benchmarking optimized performance over 100 frames...
YOLO runs every 3 frames
Progress: 0/100

AttributeError: partially initialized module 'torch._dynamo' from 'c:\Users\richa\Desktop\Personal\Uni\ShenLong\venv\Lib\site-packages\torch\_dynamo\__init__.py' has no attribute 'config' (most likely due to a circular import)

In [ ]:
# ABOUTME: Health and position monitoring for Bloody Roar II with data logging
# ABOUTME: Captures health percentages and fighter coordinates with timestamps

import cv2
import numpy as np
from mss import mss
import win32gui
import time
import json
import sys
import signal
from ultralytics import YOLO
from datetime import datetime

class HealthMonitor:
    def __init__(self, window_title, show_visualization=False):
        self.window_title = window_title
        self.show_visualization = show_visualization
        self.sct = mss()
        
        # Data storage
        self.health_data = {}
        
        # YOLO for fighter detection
        print("Loading YOLO model...")
        self.model = YOLO(r'C:\Users\richa\Desktop\Personal\Uni\ShenLong\notebooks\yolov8n.pt')
        
        # Health bar parameters
        self.health_params = {
            'p1_x': 505,
            'p2_x': 1421,
            'bar_len': 400,
            'y': 155,
            'lower_bgr': np.array([0, 160, 190], dtype=np.uint8),
            'upper_bgr': np.array([20, 180, 220], dtype=np.uint8),
            'drop_per_px': 0.25
        }
        
        # Get window handle
        self.hwnd = win32gui.FindWindow(None, self.window_title)
        if not self.hwnd:
            raise RuntimeError(f"Window not found: {self.window_title}")
            
        # Window dimensions
        rect = win32gui.GetClientRect(self.hwnd)
        self.left, self.top = win32gui.ClientToScreen(self.hwnd, (0, 0))
        self.width = rect[2]
        self.height = rect[3]
        
        # Position tracking
        self.last_p1_pos = None
        self.last_p2_pos = None
        
        # Setup signal handler for clean exit
        signal.signal(signal.SIGINT, self.signal_handler)
        
    def signal_handler(self, sig, frame):
        print("\nStopping monitor and saving data...")
        self.save_data()
        print(f"Data saved to health_data.txt ({len(self.health_data)} entries)")
        sys.exit(0)
        
    def detect_health(self):
        """Detect health bars using color detection."""
        # ROIs for health bars
        roi_p1 = {
            'left': self.left + self.health_params['p1_x'],
            'top': self.top + self.health_params['y'],
            'width': self.health_params['bar_len'],
            'height': 1,
        }
        
        roi_p2 = {
            'left': self.left + (self.health_params['p2_x'] - self.health_params['bar_len']),
            'top': self.top + self.health_params['y'],
            'width': self.health_params['bar_len'],
            'height': 1,
        }
        
        # Capture health bars
        raw_p1 = self.sct.grab(roi_p1)
        strip_p1 = np.array(raw_p1)[:, :, :3]
        b1, g1, r1 = strip_p1[0].T
        
        raw_p2 = self.sct.grab(roi_p2)
        strip_p2 = np.array(raw_p2)[:, :, :3]
        b2, g2, r2 = strip_p2[0].T
        
        # Process Player 1 (left to right)
        mask_p1 = (
            (r1 >= self.health_params['lower_bgr'][2]) & 
            (r1 <= self.health_params['upper_bgr'][2]) &
            (g1 >= self.health_params['lower_bgr'][1]) & 
            (g1 <= self.health_params['upper_bgr'][1]) &
            (b1 >= self.health_params['lower_bgr'][0]) & 
            (b1 <= self.health_params['upper_bgr'][0])
        )
        non_yellow_p1 = np.nonzero(~mask_p1)[0]
        last_idx_p1 = non_yellow_p1.max() if non_yellow_p1.size else -1
        drop_pixels_p1 = max(0, last_idx_p1 + 1)
        life_pct_p1 = 100.0 - (drop_pixels_p1 * self.health_params['drop_per_px'])
        life_pct_p1 = np.clip(life_pct_p1, 0.0, 100.0)
        
        # Process Player 2 (right to left)
        mask_p2 = (
            (r2 >= self.health_params['lower_bgr'][2]) & 
            (r2 <= self.health_params['upper_bgr'][2]) &
            (g2 >= self.health_params['lower_bgr'][1]) & 
            (g2 <= self.health_params['upper_bgr'][1]) &
            (b2 >= self.health_params['lower_bgr'][0]) & 
            (b2 <= self.health_params['upper_bgr'][0])
        )
        non_yellow_p2 = np.nonzero(~mask_p2)[0]
        last_idx_p2 = non_yellow_p2.min() if non_yellow_p2.size else self.health_params['bar_len']
        drop_pixels_p2 = self.health_params['bar_len'] - last_idx_p2
        life_pct_p2 = 100.0 - (drop_pixels_p2 * self.health_params['drop_per_px'])
        life_pct_p2 = np.clip(life_pct_p2, 0.0, 100.0)
        
        return life_pct_p1, life_pct_p2
    
    def detect_fighters(self, image):
        """Detect fighter positions using YOLO."""
        results = self.model(image, classes=[0], conf=0.3, verbose=False)
        
        fighters = []
        for r in results:
            if r.boxes is not None:
                for box in r.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    center_x = int((x1 + x2) / 2)
                    center_y = int((y1 + y2) / 2)
                    
                    # Skip if too high (UI elements)
                    if center_y > 100:
                        fighters.append({
                            'center': (center_x, center_y),
                            'box': (int(x1), int(y1), int(x2), int(y2))
                        })
        
        # Sort by x position
        fighters.sort(key=lambda f: f['center'][0])
        
        # Assign to players
        p1_pos = None
        p2_pos = None
        
        if len(fighters) >= 2:
            p1_pos = fighters[0]['center']
            p2_pos = fighters[1]['center']
        elif len(fighters) == 1:
            pos = fighters[0]['center']
            if self.last_p1_pos and self.last_p2_pos:
                dist_to_p1 = abs(pos[0] - self.last_p1_pos[0])
                dist_to_p2 = abs(pos[0] - self.last_p2_pos[0])
                if dist_to_p1 < dist_to_p2:
                    p1_pos = pos
                else:
                    p2_pos = pos
            else:
                if pos[0] < self.width // 2:
                    p1_pos = pos
                else:
                    p2_pos = pos
        
        # Update last known positions
        if p1_pos:
            self.last_p1_pos = p1_pos
        if p2_pos:
            self.last_p2_pos = p2_pos
        
        return p1_pos, p2_pos
    
    def get_game_state(self):
        """Get current game state."""
        # Capture screen
        monitor = {
            'left': self.left,
            'top': self.top,
            'width': self.width,
            'height': self.height
        }
        
        screenshot = np.array(self.sct.grab(monitor))
        image = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
        
        # Get positions and health
        p1_pos, p2_pos = self.detect_fighters(image)
        p1_health, p2_health = self.detect_health()
        
        # Calculate distance
        distance = 0
        if p1_pos and p2_pos:
            distance = abs(p1_pos[0] - p2_pos[0])
        
        return {
            'p1_pos': p1_pos,
            'p2_pos': p2_pos,
            'p1_health': p1_health,
            'p2_health': p2_health,
            'distance': distance,
            'frame': image if self.show_visualization else None
        }
    
    def save_data(self):
        """Save collected data to text file."""
        with open('health_data.txt', 'w') as f:
            f.write("Bloody Roar II Health Data Log\n")
            f.write("=" * 50 + "\n\n")
            
            for timestamp, data in sorted(self.health_data.items()):
                f.write(f"Time: {timestamp}\n")
                f.write(f"P1 X: {data['p1_x']}\n")
                f.write(f"P1 Y: {data['p1_y']}\n")
                f.write(f"P1 Health: {data['p1_health']:.1f}%\n")
                f.write(f"P2 X: {data['p2_x']}\n")
                f.write(f"P2 Y: {data['p2_y']}\n")
                f.write(f"P2 Health: {data['p2_health']:.1f}%\n")
                f.write(f"Distance: {data['distance']}\n")
                f.write("-" * 30 + "\n")
    
    def run(self):
        """Main monitoring loop."""
        print(f"Health monitor started (visualization: {'ON' if self.show_visualization else 'OFF'})")
        print("Press Ctrl+C to stop and save data")
        
        if self.show_visualization:
            cv2.namedWindow('Health Monitor', cv2.WINDOW_NORMAL)
            cv2.resizeWindow('Health Monitor', self.width // 2, self.height // 2)
        
        frame_count = 0
        
        try:
            while True:
                # Get game state
                state = self.get_game_state()
                
                # Store data with timestamp in obs format
                timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
                self.health_data[timestamp] = {
                    'p1_x': state['p1_pos'][0] if state['p1_pos'] else 0,
                    'p1_y': state['p1_pos'][1] if state['p1_pos'] else 0,
                    'p1_health': state['p1_health'],
                    'p2_x': state['p2_pos'][0] if state['p2_pos'] else 0,
                    'p2_y': state['p2_pos'][1] if state['p2_pos'] else 0,
                    'p2_health': state['p2_health'],
                    'distance': state['distance']
                }
                
                frame_count += 1
                
                # Optional visualization
                if self.show_visualization and state['frame'] is not None:
                    display = state['frame'].copy()
                    
                    # Draw positions
                    if state['p1_pos']:
                        x, y = state['p1_pos']
                        cv2.circle(display, (x, y), 5, (0, 255, 0), -1)
                        cv2.putText(display, f"P1: {state['p1_health']:.1f}%", 
                                   (x-50, y-30), cv2.FONT_HERSHEY_SIMPLEX, 
                                   0.7, (0, 255, 0), 2)
                    
                    if state['p2_pos']:
                        x, y = state['p2_pos']
                        cv2.circle(display, (x, y), 5, (0, 0, 255), -1)
                        cv2.putText(display, f"P2: {state['p2_health']:.1f}%", 
                                   (x-50, y-30), cv2.FONT_HERSHEY_SIMPLEX, 
                                   0.7, (0, 0, 255), 2)
                    
                    # Show frame count
                    cv2.putText(display, f"Frames: {frame_count}", 
                               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                               0.7, (255, 255, 255), 2)
                    
                    cv2.imshow('Health Monitor', display)
                    
                    key = cv2.waitKey(1) & 0xFF
                    if key == ord('q'):
                        break
                
                # Small delay to prevent excessive CPU usage
                time.sleep(0.001)
                
        except KeyboardInterrupt:
            pass
        
        finally:
            if self.show_visualization:
                cv2.destroyAllWindows()
            self.save_data()
            print(f"\nData saved to health_data.txt ({len(self.health_data)} entries)")


if __name__ == "__main__":
    WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"
    
    # Check for visualization flag
    show_vis = len(sys.argv) > 1 and sys.argv[1] == "--vis"
    
    try:
        monitor = HealthMonitor(WINDOW_TITLE, show_visualization=show_vis)
        monitor.run()
    except RuntimeError as e:
        print(f"Error: {e}")
        sys.exit(1)

In [4]:
# ABOUTME: Health and position monitoring for Bloody Roar II with data logging
# ABOUTME: Captures health percentages and fighter coordinates with timestamps

import cv2
import numpy as np
from mss import mss
import win32gui
import time
import json
import sys
import signal
from ultralytics import YOLO
from datetime import datetime

class HealthMonitor:
    def __init__(self, window_title, show_visualization=False):
        self.window_title = window_title
        self.show_visualization = show_visualization
        self.sct = mss()
        
        # Data storage
        self.health_data = {}
        
        # YOLO for fighter detection
        print("Loading YOLO model...")
        self.model = YOLO(r'C:\Users\richa\Desktop\Personal\Uni\ShenLong\notebooks\yolov8n.pt')
        
        # Health bar parameters
        self.health_params = {
            'p1_x': 505,
            'p2_x': 1421,
            'bar_len': 400,
            'y': 155,
            'lower_bgr': np.array([0, 160, 190], dtype=np.uint8),
            'upper_bgr': np.array([20, 180, 220], dtype=np.uint8),
            'drop_per_px': 0.25
        }
        
        # Get window handle
        self.hwnd = win32gui.FindWindow(None, self.window_title)
        if not self.hwnd:
            raise RuntimeError(f"Window not found: {self.window_title}")
            
        # Window dimensions
        rect = win32gui.GetClientRect(self.hwnd)
        self.left, self.top = win32gui.ClientToScreen(self.hwnd, (0, 0))
        self.width = rect[2]
        self.height = rect[3]
        
        # Position tracking
        self.last_p1_pos = None
        self.last_p2_pos = None
        
        # Setup signal handler for clean exit
        signal.signal(signal.SIGINT, self.signal_handler)
        
    def signal_handler(self, sig, frame):
        print("\nStopping monitor and saving data...")
        self.save_data()
        print(f"Data saved to health_data.txt ({len(self.health_data)} entries)")
        sys.exit(0)
        
    def detect_health(self):
        """Detect health bars using color detection."""
        # ROIs for health bars
        roi_p1 = {
            'left': self.left + self.health_params['p1_x'],
            'top': self.top + self.health_params['y'],
            'width': self.health_params['bar_len'],
            'height': 1,
        }
        
        roi_p2 = {
            'left': self.left + (self.health_params['p2_x'] - self.health_params['bar_len']),
            'top': self.top + self.health_params['y'],
            'width': self.health_params['bar_len'],
            'height': 1,
        }
        
        # Capture health bars
        raw_p1 = self.sct.grab(roi_p1)
        strip_p1 = np.array(raw_p1)[:, :, :3]
        b1, g1, r1 = strip_p1[0].T
        
        raw_p2 = self.sct.grab(roi_p2)
        strip_p2 = np.array(raw_p2)[:, :, :3]
        b2, g2, r2 = strip_p2[0].T
        
        # Process Player 1 (left to right)
        mask_p1 = (
            (r1 >= self.health_params['lower_bgr'][2]) & 
            (r1 <= self.health_params['upper_bgr'][2]) &
            (g1 >= self.health_params['lower_bgr'][1]) & 
            (g1 <= self.health_params['upper_bgr'][1]) &
            (b1 >= self.health_params['lower_bgr'][0]) & 
            (b1 <= self.health_params['upper_bgr'][0])
        )
        non_yellow_p1 = np.nonzero(~mask_p1)[0]
        last_idx_p1 = non_yellow_p1.max() if non_yellow_p1.size else -1
        drop_pixels_p1 = max(0, last_idx_p1 + 1)
        life_pct_p1 = 100.0 - (drop_pixels_p1 * self.health_params['drop_per_px'])
        life_pct_p1 = np.clip(life_pct_p1, 0.0, 100.0)
        
        # Process Player 2 (right to left)
        mask_p2 = (
            (r2 >= self.health_params['lower_bgr'][2]) & 
            (r2 <= self.health_params['upper_bgr'][2]) &
            (g2 >= self.health_params['lower_bgr'][1]) & 
            (g2 <= self.health_params['upper_bgr'][1]) &
            (b2 >= self.health_params['lower_bgr'][0]) & 
            (b2 <= self.health_params['upper_bgr'][0])
        )
        non_yellow_p2 = np.nonzero(~mask_p2)[0]
        last_idx_p2 = non_yellow_p2.min() if non_yellow_p2.size else self.health_params['bar_len']
        drop_pixels_p2 = self.health_params['bar_len'] - last_idx_p2
        life_pct_p2 = 100.0 - (drop_pixels_p2 * self.health_params['drop_per_px'])
        life_pct_p2 = np.clip(life_pct_p2, 0.0, 100.0)
        
        return life_pct_p1, life_pct_p2
    
    def detect_fighters(self, image):
        """Detect fighter positions using YOLO."""
        results = self.model(image, classes=[0], conf=0.3, verbose=False)
        
        fighters = []
        for r in results:
            if r.boxes is not None:
                for box in r.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    center_x = int((x1 + x2) / 2)
                    center_y = int((y1 + y2) / 2)
                    
                    # Skip if too high (UI elements)
                    if center_y > 100:
                        fighters.append({
                            'center': (center_x, center_y),
                            'box': (int(x1), int(y1), int(x2), int(y2))
                        })
        
        # Sort by x position
        fighters.sort(key=lambda f: f['center'][0])
        
        # Assign to players
        p1_pos = None
        p2_pos = None
        
        if len(fighters) >= 2:
            p1_pos = fighters[0]['center']
            p2_pos = fighters[1]['center']
        elif len(fighters) == 1:
            pos = fighters[0]['center']
            if self.last_p1_pos and self.last_p2_pos:
                dist_to_p1 = abs(pos[0] - self.last_p1_pos[0])
                dist_to_p2 = abs(pos[0] - self.last_p2_pos[0])
                if dist_to_p1 < dist_to_p2:
                    p1_pos = pos
                else:
                    p2_pos = pos
            else:
                if pos[0] < self.width // 2:
                    p1_pos = pos
                else:
                    p2_pos = pos
        
        # Update last known positions
        if p1_pos:
            self.last_p1_pos = p1_pos
        if p2_pos:
            self.last_p2_pos = p2_pos
        
        return p1_pos, p2_pos
    
    def get_game_state(self):
        """Get current game state."""
        # Capture screen
        monitor = {
            'left': self.left,
            'top': self.top,
            'width': self.width,
            'height': self.height
        }
        
        screenshot = np.array(self.sct.grab(monitor))
        image = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
        
        # Get positions and health
        p1_pos, p2_pos = self.detect_fighters(image)
        p1_health, p2_health = self.detect_health()
        
        # Calculate distance
        distance = 0
        if p1_pos and p2_pos:
            distance = abs(p1_pos[0] - p2_pos[0])
        
        return {
            'p1_pos': p1_pos,
            'p2_pos': p2_pos,
            'p1_health': p1_health,
            'p2_health': p2_health,
            'distance': distance,
            'frame': image if self.show_visualization else None
        }
    
    def save_data(self):
        """Save collected data to text file."""
        with open('health_data.txt', 'w') as f:
            f.write("Bloody Roar II Health Data Log\n")
            f.write("=" * 50 + "\n\n")
            
            for timestamp, data in sorted(self.health_data.items()):
                f.write(f"Time: {timestamp}\n")
                f.write(f"P1 X: {data['p1_x']}\n")
                f.write(f"P1 Y: {data['p1_y']}\n")
                f.write(f"P1 Health: {data['p1_health']:.1f}%\n")
                f.write(f"P2 X: {data['p2_x']}\n")
                f.write(f"P2 Y: {data['p2_y']}\n")
                f.write(f"P2 Health: {data['p2_health']:.1f}%\n")
                f.write(f"Distance: {data['distance']}\n")
                f.write("-" * 30 + "\n")
    
    def run(self):
        """Main monitoring loop."""
        print(f"Health monitor started (visualization: {'ON' if self.show_visualization else 'OFF'})")
        print("Press Ctrl+C to stop and save data")
        
        if self.show_visualization:
            cv2.namedWindow('Health Monitor', cv2.WINDOW_NORMAL)
            cv2.resizeWindow('Health Monitor', self.width // 2, self.height // 2)
        
        frame_count = 0
        
        try:
            while True:
                # Get game state
                state = self.get_game_state()
                
                # Store data with timestamp in obs format
                timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
                self.health_data[timestamp] = {
                    'p1_x': state['p1_pos'][0] if state['p1_pos'] else 0,
                    'p1_y': state['p1_pos'][1] if state['p1_pos'] else 0,
                    'p1_health': state['p1_health'],
                    'p2_x': state['p2_pos'][0] if state['p2_pos'] else 0,
                    'p2_y': state['p2_pos'][1] if state['p2_pos'] else 0,
                    'p2_health': state['p2_health'],
                    'distance': state['distance']
                }
                
                frame_count += 1
                
                # Optional visualization
                if self.show_visualization and state['frame'] is not None:
                    display = state['frame'].copy()
                    
                    # Draw positions
                    if state['p1_pos']:
                        x, y = state['p1_pos']
                        cv2.circle(display, (x, y), 5, (0, 255, 0), -1)
                        cv2.putText(display, f"P1: {state['p1_health']:.1f}%", 
                                   (x-50, y-30), cv2.FONT_HERSHEY_SIMPLEX, 
                                   0.7, (0, 255, 0), 2)
                    
                    if state['p2_pos']:
                        x, y = state['p2_pos']
                        cv2.circle(display, (x, y), 5, (0, 0, 255), -1)
                        cv2.putText(display, f"P2: {state['p2_health']:.1f}%", 
                                   (x-50, y-30), cv2.FONT_HERSHEY_SIMPLEX, 
                                   0.7, (0, 0, 255), 2)
                    
                    # Show frame count
                    cv2.putText(display, f"Frames: {frame_count}", 
                               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                               0.7, (255, 255, 255), 2)
                    
                    cv2.imshow('Health Monitor', display)
                    
                    key = cv2.waitKey(1) & 0xFF
                    if key == ord('q'):
                        break
                
                # Small delay to prevent excessive CPU usage
                time.sleep(0.001)
                
        except KeyboardInterrupt:
            pass
        
        finally:
            if self.show_visualization:
                cv2.destroyAllWindows()
            self.save_data()
            print(f"\nData saved to health_data.txt ({len(self.health_data)} entries)")


if __name__ == "__main__":
    WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"
    
    # Check for visualization flag
    show_vis = len(sys.argv) > 1 and sys.argv[1] == "--vis"
    
    try:
        monitor = HealthMonitor(WINDOW_TITLE, show_visualization=show_vis)
        monitor.run()
    except RuntimeError as e:
        print(f"Error: {e}")
        sys.exit(1)

Loading YOLO model...
Health monitor started (visualization: OFF)
Press Ctrl+C to stop and save data

Stopping monitor and saving data...
Data saved to health_data.txt (0 entries)

Data saved to health_data.txt (0 entries)


SystemExit: 0

c:\Users\richa\Desktop\Personal\Uni\ShenLong\venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# ABOUTME: Template capture POC for 2 characters in Bloody Roar II
# ABOUTME: Simple interactive tool to capture 4 templates for speed testing

import cv2
import numpy as np
from mss import mss
import win32gui
import os
import json
from datetime import datetime

class TemplateCaptureROC:
    def __init__(self, window_title):
        self.window_title = window_title
        self.sct = mss()
        
        # Get window handle and dimensions
        self.hwnd = win32gui.FindWindow(None, self.window_title)
        if not self.hwnd:
            raise RuntimeError(f"Window not found: {self.window_title}")
            
        rect = win32gui.GetClientRect(self.hwnd)
        self.left, self.top = win32gui.ClientToScreen(self.hwnd, (0, 0))
        self.width = rect[2]
        self.height = rect[3]
        
        # Create templates directory
        self.templates_dir = "templates"
        if not os.path.exists(self.templates_dir):
            os.makedirs(self.templates_dir)
            
        # Template capture settings
        self.template_size = (80, 120)  # Standard template size
        self.roi_selector = ROISelector()
        
        # Characters to capture (POC - only 2 characters)
        self.characters = ["character1", "character2"]  # You'll specify actual names
        self.forms = ["human", "beast"]
        
        # Progress tracking
        self.capture_progress = {}
        self.load_progress()
        
        print(f"Template Capture POC initialized")
        print(f"Game window: {self.width}x{self.height}")
        print(f"Templates will be saved to: {self.templates_dir}/")
        print(f"Standard template size: {self.template_size}")
        
    def load_progress(self):
        """Load capture progress from file."""
        progress_file = os.path.join(self.templates_dir, "capture_progress.json")
        if os.path.exists(progress_file):
            with open(progress_file, 'r') as f:
                self.capture_progress = json.load(f)
        else:
            # Initialize progress tracking
            self.capture_progress = {}
            for char in self.characters:
                self.capture_progress[char] = {}
                for form in self.forms:
                    self.capture_progress[char][form] = False
    
    def save_progress(self):
        """Save capture progress to file."""
        progress_file = os.path.join(self.templates_dir, "capture_progress.json")
        with open(progress_file, 'w') as f:
            json.dump(self.capture_progress, f, indent=2)
    
    def capture_frame(self):
        """Capture current game frame."""
        monitor = {
            'left': self.left,
            'top': self.top,
            'width': self.width,
            'height': self.height
        }
        
        screenshot = np.array(self.sct.grab(monitor))
        frame = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
        return frame
    
    def get_template_filename(self, character, form):
        """Get template filename for character and form."""
        return os.path.join(self.templates_dir, f"{character}_{form}.png")
    
    def template_exists(self, character, form):
        """Check if template already exists."""
        filename = self.get_template_filename(character, form)
        return os.path.exists(filename)
    
    def capture_template(self, character, form):
        """Capture a single template for character and form."""
        print(f"\n=== Capturing {character} - {form} form ===")
        
        filename = self.get_template_filename(character, form)
        
        # Check if already captured
        if self.template_exists(character, form):
            print(f"Template already exists: {filename}")
            choice = input("Overwrite? (y/n): ").lower()
            if choice != 'y':
                return True
        
        print(f"Instructions:")
        print(f"1. In the game, select {character}")
        print(f"2. Enter training mode")
        print(f"3. {'Transform to beast form' if form == 'beast' else 'Stay in human form'}")
        print(f"4. Pause the game with the fighter in neutral stance")
        print(f"5. Press ENTER when ready to capture...")
        
        input()  # Wait for user to set up
        
        # Capture frame
        frame = self.capture_frame()
        
        # Let user select ROI
        print("Select the fighter region by clicking and dragging...")
        roi = self.roi_selector.select_roi(frame, f"Select {character} ({form})")
        
        if roi is None:
            print("Template capture cancelled")
            return False
        
        # Extract and process template
        x, y, w, h = roi
        template = frame[y:y+h, x:x+w]
        
        # Resize to standard size
        template_resized = cv2.resize(template, self.template_size)
        
        # Save template
        success = cv2.imwrite(filename, template_resized)
        
        if success:
            print(f"Template saved: {filename}")
            self.capture_progress[character][form] = True
            self.save_progress()
            
            # Show preview
            cv2.imshow(f"Template Preview - {character} {form}", template_resized)
            cv2.waitKey(2000)  # Show for 2 seconds
            cv2.destroyAllWindows()
            
            return True
        else:
            print(f"Failed to save template: {filename}")
            return False
    
    def show_progress(self):
        """Show capture progress."""
        print("\n=== Capture Progress ===")
        total_templates = len(self.characters) * len(self.forms)
        completed = 0
        
        for char in self.characters:
            for form in self.forms:
                status = "✓" if self.capture_progress[char][form] else "✗"
                print(f"{status} {char} - {form}")
                if self.capture_progress[char][form]:
                    completed += 1
        
        print(f"\nCompleted: {completed}/{total_templates} templates")
        return completed == total_templates
    
    def run_capture_sequence(self):
        """Run the complete capture sequence."""
        print("=== Template Capture POC ===")
        print("This will capture 4 templates for 2 characters")
        print("Each character needs: human form + beast form")
        
        # Get character names from user
        print("\nEnter the names of the 2 characters you want to capture:")
        char1 = input("Character 1 name: ").strip()
        char2 = input("Character 2 name: ").strip()
        
        if char1 and char2:
            self.characters = [char1, char2]
            # Reinitialize progress tracking
            self.capture_progress = {}
            for char in self.characters:
                self.capture_progress[char] = {}
                for form in self.forms:
                    self.capture_progress[char][form] = False
        
        print(f"\nCapturing templates for: {', '.join(self.characters)}")
        
        # Show current progress
        self.show_progress()
        
        # Capture templates
        for character in self.characters:
            for form in self.forms:
                if not self.capture_progress[character][form]:
                    success = self.capture_template(character, form)
                    if not success:
                        print(f"Failed to capture {character} {form}")
                        return False
        
        # Final progress check
        if self.show_progress():
            print("\n✓ All templates captured successfully!")
            print(f"Templates saved in: {self.templates_dir}/")
            
            # List captured files
            print("\nCaptured files:")
            for char in self.characters:
                for form in self.forms:
                    filename = self.get_template_filename(char, form)
                    if os.path.exists(filename):
                        print(f"  - {filename}")
            
            return True
        else:
            print("\n✗ Some templates are still missing")
            return False
    
    def quick_capture_mode(self):
        """Quick capture mode - capture all templates in sequence."""
        print("\n=== Quick Capture Mode ===")
        print("This mode captures all templates with minimal interaction")
        print("Make sure you can quickly switch between characters and forms")
        
        input("Press ENTER when ready to start...")
        
        for character in self.characters:
            for form in self.forms:
                if not self.capture_progress[character][form]:
                    print(f"\n--- {character} ({form}) ---")
                    print("Set up the character and form, then press ENTER")
                    input()
                    
                    frame = self.capture_frame()
                    roi = self.roi_selector.select_roi(frame, f"{character} {form}")
                    
                    if roi:
                        x, y, w, h = roi
                        template = frame[y:y+h, x:x+w]
                        template_resized = cv2.resize(template, self.template_size)
                        
                        filename = self.get_template_filename(character, form)
                        if cv2.imwrite(filename, template_resized):
                            print(f"✓ {filename}")
                            self.capture_progress[character][form] = True
                            self.save_progress()
                        else:
                            print(f"✗ Failed to save {filename}")
                    else:
                        print("✗ Cancelled")


class ROISelector:
    def __init__(self):
        self.roi = None
        self.drawing = False
        self.start_point = None
        
    def mouse_callback(self, event, x, y, flags, param):
        """Mouse callback for ROI selection."""
        if event == cv2.EVENT_LBUTTONDOWN:
            self.drawing = True
            self.start_point = (x, y)
            
        elif event == cv2.EVENT_MOUSEMOVE:
            if self.drawing:
                # Draw rectangle while dragging
                img = param.copy()
                cv2.rectangle(img, self.start_point, (x, y), (0, 255, 0), 2)
                cv2.imshow('ROI Selection', img)
                
        elif event == cv2.EVENT_LBUTTONUP:
            self.drawing = False
            end_point = (x, y)
            
            # Calculate ROI
            x1, y1 = self.start_point
            x2, y2 = end_point
            
            # Ensure positive width and height
            x = min(x1, x2)
            y = min(y1, y2)
            w = abs(x2 - x1)
            h = abs(y2 - y1)
            
            if w > 10 and h > 10:  # Minimum size
                self.roi = (x, y, w, h)
                
                # Draw final rectangle
                img = param.copy()
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(img, "Press ENTER to confirm, ESC to cancel", 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                cv2.imshow('ROI Selection', img)
    
    def select_roi(self, frame, title="Select ROI"):
        """Select ROI from frame."""
        self.roi = None
        self.drawing = False
        
        # Create window and set mouse callback
        cv2.namedWindow('ROI Selection', cv2.WINDOW_NORMAL)
        cv2.setMouseCallback('ROI Selection', self.mouse_callback, frame)
        
        # Show frame
        display_frame = frame.copy()
        cv2.putText(display_frame, f"{title} - Click and drag to select region", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.imshow('ROI Selection', display_frame)
        
        # Wait for user interaction
        while True:
            key = cv2.waitKey(1) & 0xFF
            if key == 13:  # Enter key
                break
            elif key == 27:  # Escape key
                self.roi = None
                break
        
        cv2.destroyAllWindows()
        return self.roi


if __name__ == "__main__":
    WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"
    
    try:
        capture = TemplateCaptureROC(WINDOW_TITLE)
        
        print("Template Capture POC")
        print("Choose capture mode:")
        print("1. Guided capture (recommended)")
        print("2. Quick capture")
        
        choice = input("Enter choice (1 or 2): ").strip()
        
        if choice == "2":
            capture.quick_capture_mode()
        else:
            capture.run_capture_sequence()
            
    except RuntimeError as e:
        print(f"Error: {e}")
        print("Make sure the game window is open and visible")
    except KeyboardInterrupt:
        print("\nCapture cancelled by user")

In [ ]:
# ABOUTME: Template detection POC for fast fighter detection in Bloody Roar II
# ABOUTME: Tests speed and accuracy of template matching with 4 templates

import cv2
import numpy as np
from mss import mss
import win32gui
import time
import os
import glob
from collections import deque

class TemplateDetectorPOC:
    def __init__(self, window_title, templates_dir="templates"):
        self.window_title = window_title
        self.templates_dir = templates_dir
        self.sct = mss()
        
        # Get window handle and dimensions
        self.hwnd = win32gui.FindWindow(None, self.window_title)
        if not self.hwnd:
            raise RuntimeError(f"Window not found: {self.window_title}")
            
        rect = win32gui.GetClientRect(self.hwnd)
        self.left, self.top = win32gui.ClientToScreen(self.hwnd, (0, 0))
        self.width = rect[2]
        self.height = rect[3]
        
        # Template matching parameters
        self.match_threshold = 0.6  # Minimum confidence threshold
        self.scales = [0.8, 0.9, 1.0, 1.1, 1.2]  # Multi-scale detection
        
        # Search regions to improve speed
        self.search_regions = {
            'p1': (400, 1000, 200, 800),  # x1, x2, y1, y2 for left side
            'p2': (800, 1500, 200, 800)   # x1, x2, y1, y2 for right side
        }
        
        # Position tracking for P1/P2 assignment
        self.position_history = deque(maxlen=10)
        self.last_positions = {'p1': None, 'p2': None}
        
        # Performance tracking
        self.detection_times = []
        self.template_match_times = []
        
        # Load templates
        self.templates = self.load_templates()
        
        # Health detection (keep from original)
        self.health_params = {
            'p1_x': 505,
            'p2_x': 1421,
            'bar_len': 400,
            'y': 155,
            'lower_bgr': np.array([0, 160, 190], dtype=np.uint8),
            'upper_bgr': np.array([20, 180, 220], dtype=np.uint8),
            'drop_per_px': 0.25
        }
        
        print(f"Template Detector POC initialized")
        print(f"Game window: {self.width}x{self.height}")
        print(f"Loaded {len(self.templates)} templates")
        print(f"Search regions: P1={self.search_regions['p1']}, P2={self.search_regions['p2']}")
        
    def load_templates(self):
        """Load all templates from templates directory."""
        templates = {}
        
        if not os.path.exists(self.templates_dir):
            print(f"Templates directory not found: {self.templates_dir}")
            return templates
        
        # Find all PNG files in templates directory
        template_files = glob.glob(os.path.join(self.templates_dir, "*.png"))
        
        for template_file in template_files:
            # Extract template name from filename
            template_name = os.path.splitext(os.path.basename(template_file))[0]
            
            # Load template
            template = cv2.imread(template_file)
            if template is not None:
                templates[template_name] = template
                print(f"Loaded template: {template_name} ({template.shape[1]}x{template.shape[0]})")
            else:
                print(f"Failed to load template: {template_file}")
        
        if not templates:
            print("No templates loaded! Run template_capture_poc.py first")
        
        return templates
    
    def detect_health(self):
        """Detect health bars using existing method."""
        try:
            # ROIs for health bars
            roi_p1 = {
                'left': self.left + self.health_params['p1_x'],
                'top': self.top + self.health_params['y'],
                'width': self.health_params['bar_len'],
                'height': 1,
            }
            
            roi_p2 = {
                'left': self.left + (self.health_params['p2_x'] - self.health_params['bar_len']),
                'top': self.top + self.health_params['y'],
                'width': self.health_params['bar_len'],
                'height': 1,
            }
            
            # Capture and process health bars
            raw_p1 = self.sct.grab(roi_p1)
            strip_p1 = np.array(raw_p1)[:, :, :3]
            b1, g1, r1 = strip_p1[0].T
            
            raw_p2 = self.sct.grab(roi_p2)
            strip_p2 = np.array(raw_p2)[:, :, :3]
            b2, g2, r2 = strip_p2[0].T
            
            # Process P1 health
            mask_p1 = (
                (r1 >= self.health_params['lower_bgr'][2]) & 
                (r1 <= self.health_params['upper_bgr'][2]) &
                (g1 >= self.health_params['lower_bgr'][1]) & 
                (g1 <= self.health_params['upper_bgr'][1]) &
                (b1 >= self.health_params['lower_bgr'][0]) & 
                (b1 <= self.health_params['upper_bgr'][0])
            )
            non_yellow_p1 = np.nonzero(~mask_p1)[0]
            last_idx_p1 = non_yellow_p1.max() if non_yellow_p1.size else -1
            drop_pixels_p1 = max(0, last_idx_p1 + 1)
            life_pct_p1 = 100.0 - (drop_pixels_p1 * self.health_params['drop_per_px'])
            life_pct_p1 = np.clip(life_pct_p1, 0.0, 100.0)
            
            # Process P2 health
            mask_p2 = (
                (r2 >= self.health_params['lower_bgr'][2]) & 
                (r2 <= self.health_params['upper_bgr'][2]) &
                (g2 >= self.health_params['lower_bgr'][1]) & 
                (g2 <= self.health_params['upper_bgr'][1]) &
                (b2 >= self.health_params['lower_bgr'][0]) & 
                (b2 <= self.health_params['upper_bgr'][0])
            )
            non_yellow_p2 = np.nonzero(~mask_p2)[0]
            last_idx_p2 = non_yellow_p2.min() if non_yellow_p2.size else self.health_params['bar_len']
            drop_pixels_p2 = self.health_params['bar_len'] - last_idx_p2
            life_pct_p2 = 100.0 - (drop_pixels_p2 * self.health_params['drop_per_px'])
            life_pct_p2 = np.clip(life_pct_p2, 0.0, 100.0)
            
            return life_pct_p1, life_pct_p2
            
        except Exception as e:
            print(f"Health detection error: {e}")
            return 0.0, 0.0
    
    def match_template_multiscale(self, image, template, scales=None):
        """Match template at multiple scales."""
        if scales is None:
            scales = self.scales
        
        best_match = None
        best_score = 0
        best_scale = 1.0
        
        template_match_start = time.perf_counter()
        
        for scale in scales:
            # Scale template
            scaled_template = cv2.resize(template, None, fx=scale, fy=scale)
            
            # Skip if template is larger than image
            if scaled_template.shape[0] > image.shape[0] or scaled_template.shape[1] > image.shape[1]:
                continue
            
            # Template matching
            result = cv2.matchTemplate(image, scaled_template, cv2.TM_CCOEFF_NORMED)
            _, max_val, _, max_loc = cv2.minMaxLoc(result)
            
            if max_val > best_score:
                best_score = max_val
                best_match = max_loc
                best_scale = scale
        
        template_match_time = (time.perf_counter() - template_match_start) * 1000
        self.template_match_times.append(template_match_time)
        
        return best_match, best_score, best_scale
    
    def detect_fighters_template_matching(self, frame):
        """Detect fighters using template matching."""
        detection_start = time.perf_counter()
        
        detections = []
        
        # Search in both regions
        for region_name, (x1, x2, y1, y2) in self.search_regions.items():
            # Extract search region
            search_roi = frame[y1:y2, x1:x2]
            
            # Try all templates in this region
            region_detections = []
            
            for template_name, template in self.templates.items():
                match_loc, score, scale = self.match_template_multiscale(search_roi, template)
                
                if match_loc and score > self.match_threshold:
                    # Convert to full frame coordinates
                    full_x = match_loc[0] + x1
                    full_y = match_loc[1] + y1
                    
                    # Calculate center point
                    template_h, template_w = template.shape[:2]
                    center_x = full_x + int(template_w * scale / 2)
                    center_y = full_y + int(template_h * scale / 2)
                    
                    region_detections.append({
                        'center': (center_x, center_y),
                        'template': template_name,
                        'score': score,
                        'scale': scale,
                        'region': region_name,
                        'bbox': (full_x, full_y, int(template_w * scale), int(template_h * scale))
                    })
            
            # Get best detection in this region
            if region_detections:
                best_detection = max(region_detections, key=lambda x: x['score'])
                detections.append(best_detection)
        
        # Assign to P1 and P2
        p1_pos, p2_pos = self.assign_players(detections)
        
        detection_time = (time.perf_counter() - detection_start) * 1000
        self.detection_times.append(detection_time)
        
        return p1_pos, p2_pos, detections
    
    def assign_players(self, detections):
        """Assign detections to P1 and P2."""
        if not detections:
            return None, None
        
        if len(detections) == 1:
            # Single detection - assign based on region or previous position
            detection = detections[0]
            pos = detection['center']
            
            if detection['region'] == 'p1':
                return pos, None
            elif detection['region'] == 'p2':
                return None, pos
            else:
                # Fallback: assign based on screen position
                if pos[0] < self.width // 2:
                    return pos, None
                else:
                    return None, pos
        
        # Multiple detections - assign based on x-position
        detections.sort(key=lambda x: x['center'][0])
        
        p1_pos = detections[0]['center']
        p2_pos = detections[1]['center'] if len(detections) > 1 else None
        
        return p1_pos, p2_pos
    
    def get_game_state(self):
        """Get current game state using template matching."""
        frame_start = time.perf_counter()
        
        # Capture screen
        monitor = {
            'left': self.left,
            'top': self.top,
            'width': self.width,
            'height': self.height
        }
        
        screenshot = np.array(self.sct.grab(monitor))
        frame = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
        
        # Detect fighters
        p1_pos, p2_pos, detections = self.detect_fighters_template_matching(frame)
        
        # Update position history
        self.last_positions['p1'] = p1_pos
        self.last_positions['p2'] = p2_pos
        
        # Get health
        p1_health, p2_health = self.detect_health()
        
        # Calculate distance
        distance = 0
        if p1_pos and p2_pos:
            distance = abs(p1_pos[0] - p2_pos[0])
        
        frame_time = (time.perf_counter() - frame_start) * 1000
        
        return {
            'p1_pos': p1_pos,
            'p2_pos': p2_pos,
            'p1_health': p1_health,
            'p2_health': p2_health,
            'distance': distance,
            'frame': frame,
            'detections': detections,
            'frame_time_ms': frame_time,
            'detection_time_ms': self.detection_times[-1] if self.detection_times else 0,
            'template_match_time_ms': sum(self.template_match_times[-len(self.templates):]) if self.template_match_times else 0
        }
    
    def run_speed_test(self, num_frames=100):
        """Run speed test for specified number of frames."""
        print(f"\nRunning speed test for {num_frames} frames...")
        
        # Reset timing arrays
        self.detection_times = []
        self.template_match_times = []
        
        frame_times = []
        
        print("Starting speed test...")
        for i in range(num_frames):
            state = self.get_game_state()
            frame_times.append(state['frame_time_ms'])
            
            if i % 10 == 0:
                current_avg = np.mean(frame_times[-10:]) if frame_times else 0
                print(f"Frame {i}/{num_frames} - Avg time: {current_avg:.2f}ms")
        
        # Calculate statistics
        avg_frame_time = np.mean(frame_times)
        avg_detection_time = np.mean(self.detection_times)
        avg_template_match_time = np.mean(self.template_match_times)
        std_frame_time = np.std(frame_times)
        fps = 1000 / avg_frame_time if avg_frame_time > 0 else 0
        
        print(f"\n=== TEMPLATE MATCHING SPEED TEST RESULTS ===")
        print(f"Templates loaded: {len(self.templates)}")
        print(f"Total frames processed: {num_frames}")
        print(f"Average frame time: {avg_frame_time:.2f}ms (±{std_frame_time:.2f}ms)")
        print(f"Average detection time: {avg_detection_time:.2f}ms")
        print(f"Average template matching time: {avg_template_match_time:.2f}ms")
        print(f"Average FPS: {fps:.1f}")
        print(f"Min frame time: {min(frame_times):.2f}ms")
        print(f"Max frame time: {max(frame_times):.2f}ms")
        print(f"95th percentile: {np.percentile(frame_times, 95):.2f}ms")
        
        # Compare with targets
        print(f"\n=== PERFORMANCE ANALYSIS ===")
        target_fps = 30
        target_frame_time = 1000 / target_fps
        
        if avg_frame_time <= target_frame_time:
            print(f"✓ SUCCESS: {fps:.1f} FPS exceeds target of {target_fps} FPS")
        else:
            print(f"✗ SLOW: {fps:.1f} FPS below target of {target_fps} FPS")
        
        return {
            'avg_frame_time': avg_frame_time,
            'avg_detection_time': avg_detection_time,
            'avg_template_match_time': avg_template_match_time,
            'fps': fps,
            'std_frame_time': std_frame_time,
            'min_time': min(frame_times),
            'max_time': max(frame_times),
            'success': avg_frame_time <= target_frame_time
        }
    
    def run_visualization(self):
        """Run with real-time visualization."""
        cv2.namedWindow('Template Detection POC', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Template Detection POC', self.width // 2, self.height // 2)
        
        print("Visualization started. Press 'q' to quit, 's' for speed test")
        
        try:
            while True:
                state = self.get_game_state()
                
                # Draw on main frame
                display = state['frame'].copy()
                
                # Draw search regions
                for region_name, (x1, x2, y1, y2) in self.search_regions.items():
                    color = (0, 255, 0) if region_name == 'p1' else (0, 0, 255)
                    cv2.rectangle(display, (x1, y1), (x2, y2), color, 1)
                    cv2.putText(display, region_name.upper(), (x1, y1-5), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
                
                # Draw detected fighters
                if state['p1_pos']:
                    cv2.circle(display, state['p1_pos'], 10, (0, 255, 0), -1)
                    cv2.putText(display, f"P1: {state['p1_health']:.1f}%", 
                               (state['p1_pos'][0]-50, state['p1_pos'][1]-30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                
                if state['p2_pos']:
                    cv2.circle(display, state['p2_pos'], 10, (0, 0, 255), -1)
                    cv2.putText(display, f"P2: {state['p2_health']:.1f}%", 
                               (state['p2_pos'][0]-50, state['p2_pos'][1]-30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                
                # Draw all detections
                for detection in state['detections']:
                    x, y, w, h = detection['bbox']
                    cv2.rectangle(display, (x, y), (x+w, y+h), (255, 255, 0), 2)
                    cv2.putText(display, f"{detection['template']}: {detection['score']:.2f}", 
                               (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0), 1)
                
                # Draw performance info
                cv2.putText(display, f"Frame: {state['frame_time_ms']:.1f}ms", 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(display, f"Detection: {state['detection_time_ms']:.1f}ms", 
                           (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(display, f"Template Match: {state['template_match_time_ms']:.1f}ms", 
                           (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                
                if state['distance']:
                    cv2.putText(display, f"Distance: {state['distance']}", 
                               (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                
                # Show frame
                cv2.imshow('Template Detection POC', display)
                
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('s'):
                    self.run_speed_test(50)
                    
        except KeyboardInterrupt:
            print("\nStopped by user")
        finally:
            cv2.destroyAllWindows()


if __name__ == "__main__":
    WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"
    
    try:
        print("=== Template Detection POC ===")
        print("Make sure you have captured templates first using template_capture_poc.py")
        
        detector = TemplateDetectorPOC(WINDOW_TITLE)
        
        if not detector.templates:
            print("No templates found! Please run template_capture_poc.py first")
            exit(1)
        
        # Run automated speed test first
        results = detector.run_speed_test(100)
        
        # Show results summary
        if results['success']:
            print(f"\n🎉 POC SUCCESS! Template matching achieves {results['fps']:.1f} FPS")
            print("This is fast enough for RL training!")
        else:
            print(f"\n⚠️  POC SLOW: Only {results['fps']:.1f} FPS")
            print("May need optimization or fewer templates")
        
        # Ask for visualization
        choice = input("\nShow visualization? (y/n): ").strip().lower()
        if choice == 'y':
            detector.run_visualization()
        
    except RuntimeError as e:
        print(f"Error: {e}")
        print("Make sure the game window is open and visible")
    except KeyboardInterrupt:
        print("\nTest cancelled by user")